# Code here is extremly messy, this notebook contains lot's of experimentation to understand inner working of BERT model and nuances of Huggingface library

In [ ]:
import time
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [3]:
from sklearn import metrics
import transformers
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel

In [4]:
import warnings
import logging
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.ERROR)

In [5]:
import io
import os
import random
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

In [6]:
device = 'cpu'

MAX_LEN = 512

TRAIN_BATCH_SIZE = 16

num_workers = 0

In [7]:
dtrain = pd.read_csv('../data/can_train.csv')

In [8]:
# data['labels'] = data.iloc[:, 1:].values.tolist()

In [10]:
model_path = 'emilyalsentzer/Bio_Discharge_Summary_BERT'

In [26]:
# model_path = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

In [27]:
# tokenizer = AutoTokenizer.from_pretrained(model_path, truncation=True)

In [9]:
tokenizer = BertTokenizer.from_pretrained(model_path, truncation=True)

#### sample input and tokenizer example

In [10]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [11]:
# different that BERT

In [25]:
tokenizer.convert_tokens_to_ids(["[MASK]"])

[4]

In [13]:
# sample_row = dtrain.iloc[9]
# sample_comment = sample_row.TEXT
# LABEL_COLUMNS = data.columns.tolist()[1:15]
# sample_labels = sample_row[LABEL_COLUMNS].tolist()

In [15]:
sample_labels

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [88]:
# print(sample_comment)
# print(sample_labels.to_dict())

### Truncation strats

#### regular way with head truncated till 512

In [58]:
# more detail on multiple encoding weays in bert; refer this: https://jaketae.github.io/category/common-sense/

In [16]:
enc = tokenizer.encode_plus(
  sample_comment,
  add_special_tokens=True,
  max_length=512,
  padding="max_length",  # padding will pad the shorter sentense to max length with zeroes
  return_token_type_ids=True, # [CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP] is doen for sequence classfication
                                # so token type ids can diffrentaite between two sentences
  return_attention_mask=True, # The attention mask is a binary tensor indicating the position of the
                                # padded indices so that the model does not attend to them.
  return_tensors='pt'#, # If set, will return tensors instead of list of python integers
#   truncation = True # truncation will truncate input to max length
)

enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
enc["input_ids"].shape, enc["token_type_ids"].shape, enc["attention_mask"].shape

(torch.Size([1, 2880]), torch.Size([1, 2880]), torch.Size([1, 2880]))

In [23]:
enc["input_ids"].squeeze()[:512]

tensor([    2,  5739, 22339, 18829, 16510,  9384,  3605,  1814,  1683,  8264,
         2895,  1725, 20304,  1690,  2015,  1685, 14834,  2961,  1717,  1685,
         3855,  2137,  5413,  1683,  2069,  3186,  2117,  1715,  2961,  1717,
         1685, 10786,  1690,  4310,  4334,  1715,  9384,  9832,  3328,  2430,
         2373, 14605,  7730,  2386,  1734,  9266,  1814,  2895,  1725,  3190,
         1685,  2431,  9879, 14834,  1680,  2348,  3202,  1701,  1680,  2895,
         2895,  1755,  1715, 28137,  1788,  1685,  3127,  1685, 16845, 26767,
         1838, 10166,  7263,  1026,  6272,  5422,  9056,  7956, 23750,  1026,
        16021, 17141, 11239,  1690, 10100,  1680,  2348,  1734,  6775,  1701,
         1680,  2895,  2530,  4901,  1007,  3723,  1894, 17213,  2137,  1685,
         1894, 17213, 14857,  9749,  1715, 15213,  7048, 15206,  1021,  7368,
         1732,  2614, 18766,  1026,  3723,  1680,  2348,  5990,  2105,  3723,
         1755,  1950,  1798,  1680, 21011,  1685,  6334,  1734, 

In [19]:
enc["token_type_ids"].squeeze()

tensor([0, 0, 0,  ..., 0, 0, 0])

In [20]:
enc["attention_mask"].squeeze()

tensor([1, 1, 1,  ..., 1, 1, 1])

In [64]:
print(tokenizer.convert_ids_to_tokens(enc["input_ids"].squeeze()))

['[CLS]', 'service', 'cardio', '##thoracic', 'chief', 'sick', 'transfer', 'from', 'an', 'outside', 'hospital', 'for', 'workup', 'and', 'treatment', 'of', 'endocarditis', 'chronic', '##le', 'of', 'lay', 'out', 'illness', 'an', 'year', 'old', 'man', 'with', 'chronic', '##le', 'of', 'cardiomyopathy', 'and', 'hypertension', 'along', 'with', 'sick', 'fistula', 'syndrome', 'condition', 'post', 'pacemaker', 'placement', 'who', 'was', 'transferred', 'from', 'hospital', 'for', 'management', 'of', 'conf', '##utable', 'endocarditis', 'the', 'patient', 'presented', 'to', 'the', 'hospital', 'hospital', 'on', 'with', 'ail', '##ment', 'of', 'weeks', 'of', 'pertin', '##aci', '##ous', 'intermittent', 'fever', '##s', 'arth', '##ral', '##gia', 'chi', '##ll', '##s', 'anorexia', 'sickness', 'vomiting', 'and', 'diarrhea', 'the', 'patient', 'was', 'admitted', 'to', 'the', 'hospital', 'where', 'rou', '##e', 'culture', 'de', '##uce', 'out', 'of', 'de', '##uce', 'came', 'hind', 'with', 'methicillin', 'susceptib

#### truncation strategies 2 (head + tail)

In [65]:
# Basically, if the sequence goes over the max length of 512, they took the first 128 (25% of 510)
# and the last 382 tokens (75% of 510).

# This is a valuable trick in particularly long text because it allows you to capture the most important parts of the text, 
# summary/thesis and conclusion of writing, but it is less important when your text is not that long. 

In [66]:
input_ids_list = torch.cat([enc["input_ids"].squeeze()[:256], enc["input_ids"].squeeze()[-256:]])
# adding extra dimension
input_ids_list = torch.unsqueeze(input_ids_list, 0)

In [67]:
input_ids_list.shape

torch.Size([1, 512])

In [68]:
attention_mask_list = torch.cat([enc["token_type_ids"].squeeze()[:256], enc["token_type_ids"].squeeze()[-256:]])
# adding extra dimension
attention_mask_list = torch.unsqueeze(attention_mask_list, 0)

In [69]:
attention_mask_list.shape

torch.Size([1, 512])

In [70]:
token_type_ids_list = torch.cat([enc["attention_mask"].squeeze()[:256], enc["attention_mask"].squeeze()[-256:]])
# adding extra dimension
token_type_ids_list = torch.unsqueeze(token_type_ids_list, 0)

In [71]:
token_type_ids_list.shape

torch.Size([1, 512])

#### truncation with combining all with sliding window sizing

In [72]:
enc2 = tokenizer.encode_plus(
    sample_comment,
    max_length=400,
    pad_to_max_length=True,
    add_special_tokens=True,
    return_attention_mask=True,
    return_token_type_ids=True,
    return_overflowing_tokens=True,
    return_tensors='pt',
    truncation = True)

In [73]:
enc2.keys()

dict_keys(['overflowing_tokens', 'num_truncated_tokens', 'input_ids', 'token_type_ids', 'attention_mask'])

In [74]:
enc2['overflowing_tokens'].shape, enc2['num_truncated_tokens'].shape, enc2["input_ids"].shape, enc2["token_type_ids"].shape, enc2["attention_mask"].shape

(torch.Size([1, 2480]),
 torch.Size([1]),
 torch.Size([1, 400]),
 torch.Size([1, 400]),
 torch.Size([1, 400]))

In [75]:
enc2["input_ids"].squeeze()

tensor([    2,  5739, 22339, 18829, 16510,  9384,  3605,  1814,  1683,  8264,
         2895,  1725, 20304,  1690,  2015,  1685, 14834,  2961,  1717,  1685,
         3855,  2137,  5413,  1683,  2069,  3186,  2117,  1715,  2961,  1717,
         1685, 10786,  1690,  4310,  4334,  1715,  9384,  9832,  3328,  2430,
         2373, 14605,  7730,  2386,  1734,  9266,  1814,  2895,  1725,  3190,
         1685,  2431,  9879, 14834,  1680,  2348,  3202,  1701,  1680,  2895,
         2895,  1755,  1715, 28137,  1788,  1685,  3127,  1685, 16845, 26767,
         1838, 10166,  7263,  1026,  6272,  5422,  9056,  7956, 23750,  1026,
        16021, 17141, 11239,  1690, 10100,  1680,  2348,  1734,  6775,  1701,
         1680,  2895,  2530,  4901,  1007,  3723,  1894, 17213,  2137,  1685,
         1894, 17213, 14857,  9749,  1715, 15213,  7048, 15206,  1021,  7368,
         1732,  2614, 18766,  1026,  3723,  1680,  2348,  5990,  2105,  3723,
         1755,  1950,  1798,  1680, 21011,  1685,  6334,  1734, 

In [76]:
# enc2["token_type_ids"].squeeze()

In [77]:
# enc2["attention_mask"].squeeze()

In [78]:
long_terms_token = []
input_ids_list = []
attention_mask_list = []
token_type_ids_list = []
targets_list = []

previous_input_ids = enc2["input_ids"].reshape(-1)
previous_attention_mask = enc2["attention_mask"].reshape(-1)
previous_token_type_ids = enc2["token_type_ids"].reshape(-1)
remain = enc2.get("overflowing_tokens").reshape(-1)

In [79]:
targets = torch.tensor(sample_labels, dtype=torch.int)

In [80]:
input_ids_list.append(previous_input_ids)
attention_mask_list.append(previous_attention_mask)
token_type_ids_list.append(previous_token_type_ids)
targets_list.append(targets)

In [81]:
remain = torch.tensor(remain, dtype=torch.long)
idxs = range(len(remain) + 400)
idxs = idxs[(400-30-2)
             ::(400-30-2)]
input_ids_first_overlap = previous_input_ids[-(30+1):-1]
start_token = torch.tensor([2], dtype=torch.long)
end_token = torch.tensor([3], dtype=torch.long)

for i, idx in enumerate(idxs):
    if i == 0:
        input_ids = torch.cat(
            (input_ids_first_overlap, remain[:idx]))
    elif i == len(idxs):
        input_ids = remain[idx:]
    elif previous_idx >= len(remain):
        break
    else:
        input_ids = remain[(previous_idx-30):idx]

    previous_idx = idx

    nb_token = len(input_ids) + 2
    attention_mask = torch.ones(400, dtype=torch.long)
    attention_mask[nb_token:400] = 0
    token_type_ids = torch.zeros(400, dtype=torch.long)
    input_ids = torch.cat((start_token, input_ids, end_token))
    if 400 - nb_token > 0:
        padding = torch.zeros(400-nb_token, dtype=torch.long)
        input_ids = torch.cat((input_ids, padding))

    input_ids_list.append(input_ids)
    attention_mask_list.append(attention_mask)
    token_type_ids_list.append(token_type_ids)
    targets_list.append(targets)

In [87]:
len(input_ids_list)

8

In [91]:
# print(tokenizer.convert_ids_to_tokens(enc2['overflowing_tokens'].squeeze()))

In [83]:
# 'targets': torch.tensor(self.targets[index], dtype=torch.float)

### sample model test

In [121]:
class CustomDataset(Dataset):

    def __init__(self, text, label, tokenizer, max_len, approach):
        self.tokenizer = tokenizer
        self.text = text
        self.label = label
        self.max_len = max_len
        self.trunc = approach

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        if self.trunc == 'head':

            inputs = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                return_attention_mask = True,
                truncation = True,
                return_tensors='pt'
            )
            
            ids = inputs['input_ids']
            ids = torch.squeeze(ids, 0)
            mask = inputs['attention_mask']
            mask = torch.squeeze(mask, 0)


        elif self.trunc == 'mixed':

            inputs = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                return_attention_mask = True,
                return_tensors='pt')

            ids = torch.cat([inputs["input_ids"].squeeze()[:256], inputs["input_ids"].squeeze()[-256:]])
            mask = torch.cat([inputs["attention_mask"].squeeze()[:256], inputs["attention_mask"].squeeze()[-256:]])

        
        elif self.trunc == 'tail':
            
            inputs = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                return_attention_mask = True,
                return_tensors='pt')
            
            ids = torch.cat([inputs["input_ids"].squeeze()[:1], inputs["input_ids"].squeeze()[-511:]])
            mask = torch.cat([inputs["attention_mask"].squeeze()[:1], inputs["attention_mask"].squeeze()[-512:]])

        else:
            raise NotImplementedError


        return {
            'ids': ids,
            'mask': mask,
            'targets': torch.tensor(self.label[index], dtype=torch.long)
        }

In [122]:
# class CustomDataset(Dataset):

#     def __init__(self, text, label, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.text = text
#         self.label = label
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.text)

#     def __getitem__(self, index):
#         text = str(self.text[index])
#         text = ' '.join(text.split())

#         inputs = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_attention_mask = True,
#             truncation = True
#         )
        
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']

#         return {
#             'ids': torch.tensor(ids),
#             'mask': torch.tensor(mask),
#             'targets': torch.tensor(self.label[index], dtype=torch.long)
#         }

In [123]:
training_set = CustomDataset(dtrain.TEXT.values, dtrain['LABEL'].values, tokenizer, MAX_LEN, approach = 'tail')

In [124]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                    'num_workers': num_workers,
                    'shuffle': False
                    }

In [125]:
training_loader = DataLoader(training_set, **train_params)

In [126]:
print(f'Created `torch_train_dataloader` with {len(training_loader)} batches!')

Created `torch_train_dataloader` with 94 batches!


In [127]:
sample_batch = next(iter(training_loader))

In [128]:
sample_batch["ids"].shape, sample_batch["mask"].shape

(torch.Size([16, 512]), torch.Size([16, 513]))

In [129]:
sample_batch.keys()

dict_keys(['ids', 'mask', 'targets'])

In [130]:
len(sample_batch["ids"])

16

#### Head

In [86]:
sample_batch["ids"]

tensor([[  101,  5182,   178,  ...,  1174, 24914,   102],
        [  101,  5182,   182,  ...,  1151,  1113,   102],
        [  101,  5182,   170,  ..., 10008,  4063,   102],
        ...,
        [  101,   172,  6385,  ..., 16426,  6617,   102],
        [  101,  1940, 10182,  ...,  2587,  1111,   102],
        [  101,  5182,  1260,  ...,   183,   181,   102]])

In [87]:
sample_batch["mask"]

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [88]:
sample_batch["targets"]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

#### Mixed

In [99]:
sample_batch["ids"]

tensor([[  101,  5182,   178,  ...,  2063,  1118,   102],
        [  101,  5182,   182,  ...,   182,  1181,   102],
        [  101,  5182,   170,  ...,  2063,  1118,   102],
        ...,
        [  101,   172,  6385,  ...,  1118,  2261,   102],
        [  101,  1940, 10182,  ...,  1118,  2261,   102],
        [  101,  5182,  1260,  ...,  2063,  1118,   102]])

In [100]:
sample_batch["mask"]

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [101]:
sample_batch["targets"]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

#### tail

In [131]:
sample_batch["ids"]

tensor([[  101,   185,  1186,  ...,  2063,  1118,   102],
        [  101,  1551,  1285,  ...,   182,  1181,   102],
        [  101,  1112,  1834,  ...,  2063,  1118,   102],
        ...,
        [  101,  1159,  1103,  ...,  1118,  2261,   102],
        [  101, 21256, 16516,  ...,  1118,  2261,   102],
        [  101,  1179,  1322,  ...,  2063,  1118,   102]])

In [132]:
sample_batch["mask"]

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [133]:
sample_batch["targets"]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

#### little bit about the BERT model outputs

In [ ]:
# BERT (the base model without any heads on top) outputs 2 things: last_hidden_state and pooler_output.

# last_hidden_state contains the hidden representations for each token in each sequence of the batch.
# size is (batch_size, seq_len, hidden_size)
# pooler_output contains a "representation" of each sequence in the batch, and is of size (batch_size, hidden_size). 
# What it basically does is take the hidden representation of the [CLS] token of each sequence in the batch 
# (which is a vector of size hidden_size), and then run that through the BertPooler nn.Module. 
# This consists of a linear layer followed by a Tanh activation function. 
# The weights of this linear layer are already pretrained on the next sentence prediction task. 

In [ ]:
# alternate hyphothesis (https://github.com/huggingface/transformers/issues/1328)

In [ ]:
# In the paper, they state the following about the pooler output: "This output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence."
# So it's actually better to use outputs[0], which are the hidden representations of all tokens, and take the average. 
# But what actually also works well in practice is just using the hidden representation of the [CLS] token. 
# The reason this [CLS] token is introduced is because it can be used for classification tasks. 
# You can see the hidden representation of the [CLS] token as a representation of the whole sequence (sentence).
# Since outputs[0] is of size (batch_size, seq_len, hidden_size), and 
# we only want the vector of the [CLS] token, we can obtain it by typing outputs[0][:, 0, :]. 
# You can then apply a dropout layer,
# followed by a linear layer on top of that to get 2 outputs (in case you are doing binary text classification).


# example: 
# hidden_states = outputs[0] # here outputs[1] is the 
# pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
# pooled_output = pooled_output[:, 0, :]
# pooled_output = self.dropout(pooled_output)
# # classifier of course has to be 4 * hidden_dim, because we concat 4 layers
# logits = self.classifier(pooled_output)

In [ ]:
# so that's why in the forward pass, we use outputs[1], meaning the output of the pooler layer

In [18]:
import torch.nn as nn

class BERTClass(nn.Module):
    def __init__(self, model_path):
        super(BERTClass, self).__init__()
        self.model = AutoModel.from_pretrained(model_path, output_attentions=True)
        self.l3 = torch.nn.Dropout(0.1)
        self.l4 = torch.nn.Linear(768, 2)
                        
        # nn.init.normal_(self.l4.weight, std=0.02)
        # nn.init.xavier_normal_(self.l4.weight)

    
    def forward(self, ids, token_type_ids=None, attention_mask=None, labels=None):

        _, pooled_output, _ = self.model(ids, attention_mask = attention_mask)
        pooled_output = self.l3(pooled_output)
        logits = self.l4(pooled_output)
        return logits

In [19]:
model = BERTClass(model_path)

In [31]:
loss_fn = torch.nn.CrossEntropyLoss()

In [27]:
# model

In [20]:
model_A = AutoModel.from_pretrained(model_path, return_dict=True, output_attentions = True)

In [52]:
# list(model_A.named_parameters())

In [60]:
# _model.get_input_embeddings()

In [21]:
model_B = BertForSequenceClassification.from_pretrained(model_path, return_dict=True, num_labels=2, output_attentions = True)

Some weights of the model checkpoint at emilyalsentzer/Bio_Discharge_Summary_BERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from

In [29]:
# model_B

In [22]:
model_C = BertModel.from_pretrained(model_path, return_dict=True, output_attentions = True)

In [30]:
# model_C

#### Return dict // Wrapper outputts (loss , logits)

In [32]:
op = model(sample_batch["ids"], sample_batch["mask"])

In [35]:
op #// logits

tensor([[ 1.7794, -1.1404],
        [ 1.3821, -1.2557],
        [ 1.3772, -1.1841],
        [ 1.2998, -0.7782],
        [ 1.3964, -1.7109],
        [ 1.0361, -0.9193],
        [ 1.2031, -0.6658],
        [ 1.3857, -0.8456],
        [ 1.2732, -0.8373],
        [ 1.0309, -1.2989],
        [ 0.6155, -0.2937],
        [ 0.9370, -0.9654],
        [ 1.7419, -1.2648],
        [ 1.8184, -0.9073],
        [ 1.0246, -1.2544],
        [ 1.2596, -0.7706]], grad_fn=<AddmmBackward>)

In [36]:
loss = loss_fn(op, sample_batch["targets"])

In [37]:
loss

tensor(0.4109, grad_fn=<NllLossBackward>)

In [38]:
op = model_A(sample_batch["ids"], sample_batch["mask"])

In [39]:
op.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'attentions'])

In [47]:
# op.attentions

In [40]:
op = model_B(sample_batch["ids"], sample_batch["mask"], labels = sample_batch["targets"])

In [41]:
op.keys()

odict_keys(['loss', 'logits', 'attentions'])

In [43]:
op.loss

tensor(0.8417, grad_fn=<NllLossBackward>)

In [44]:
op.logits

tensor([[-0.0818,  0.2177],
        [-0.1514,  0.2702],
        [-0.0756,  0.2701],
        [-0.0987,  0.2703],
        [-0.2117,  0.3176],
        [-0.0355,  0.2130],
        [-0.0663,  0.2552],
        [-0.0357,  0.2273],
        [-0.0066,  0.2002],
        [-0.0149,  0.2339],
        [-0.0041,  0.2480],
        [ 0.0282,  0.1816],
        [-0.0854,  0.2813],
        [-0.1345,  0.3194],
        [-0.0330,  0.2412],
        [-0.1430,  0.3049]], grad_fn=<AddmmBackward>)

In [46]:
# op.attentions

In [48]:
op = model_C(sample_batch["ids"], sample_batch["mask"])

In [49]:
op.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'attentions'])

In [47]:
# op.logits

#### No return dict

#### OP1 // Sequence Output, (B, P, H): The final hidden states outputted from the encoder in the last BERT layer
#### OP2 // Pooled Output, (B, H): The final hidden vector associated with the first input token ([CLS]) outputted from the encoder in the last BERT layer, and underwent a linear layer with tanh() activation
#### OP3 // All Hidden States, (([K+1] x (B, P, H))): Hidden states outputted from the encoders of all the BERT layers
#### OP4 // Attention Output, (([K] x (B, N, P, P))): Multi-head attention weights extracted from all the BERT layers

In [17]:
__model = BertModel.from_pretrained(model_path, output_attentions = True, output_hidden_states=True)

In [19]:
op1, op2, op3, op4 = __model(sample_batch["ids"], sample_batch["mask"])

In [41]:
op1.shape # Last hidden layer rep

torch.Size([16, 512, 768])

In [50]:
op1[0]

tensor([[-0.0463,  0.0952, -0.3890,  ..., -0.1613,  0.0261,  0.0222],
        [-0.2807, -0.5069, -0.2469,  ..., -0.0743, -0.1081, -0.0718],
        [ 0.4939,  0.0877, -0.4353,  ...,  0.3778, -0.1590, -0.2835],
        ...,
        [-0.1434, -0.0442,  0.0896,  ..., -0.0254, -0.1575, -0.2242],
        [-0.0615,  0.1393, -0.0202,  ..., -0.5599, -0.9070, -0.4680],
        [ 0.2698, -0.4495, -0.2504,  ..., -0.2859,  0.8143,  0.5545]],
       grad_fn=<SelectBackward>)

In [20]:
op2.shape # pooler output

torch.Size([16, 768])

In [ ]:
# if output_hidden_states=True
# then Hidden-states of the model at the output of each layer plus the initial embedding outputs. 
# (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).

In [44]:
len(op3)

13

In [60]:
op3[0].shape

torch.Size([16, 512, 768])

In [61]:
embedding_output = op3[0]
attention_hidden_states = op3[1:]

In [62]:
embedding_output.shape

torch.Size([16, 512, 768])

In [64]:
len(attention_hidden_states) # 12 hidden layers, each with 12 attention heads

12

#### Method borrowed from here https://github.com/kldarek/skok/blob/master/_notebooks/2020-10-27-Training%20BERT%20for%20Cyberbullying%20Detection%20-%20Part%202.ipynb

In [65]:
out = op3[-2]

In [66]:
out.shape

torch.Size([16, 512, 768])

In [70]:
out = out.permute(0,2,1)

In [72]:
out.shape

torch.Size([16, 768, 512])

In [74]:
mx = torch.nn.MaxPool1d(512)

In [75]:
out = torch.squeeze(mx(out)) #  take the max value for each feature across all tokens

In [77]:
out.shape

torch.Size([16, 768])

In [78]:
out

tensor([[1.3152, 1.1395, 1.2728,  ..., 2.1631, 2.2723, 1.6337],
        [1.4308, 1.3393, 1.7249,  ..., 2.1302, 2.0853, 1.6739],
        [1.3211, 1.6001, 0.9981,  ..., 2.0038, 1.7733, 2.0139],
        ...,
        [1.6848, 1.9105, 1.2422,  ..., 1.9123, 1.6752, 1.9428],
        [1.5303, 1.7909, 1.3823,  ..., 1.8821, 1.5183, 2.1037],
        [1.5545, 1.4240, 1.5135,  ..., 1.7933, 1.7887, 2.3944]],
       grad_fn=<SqueezeBackward0>)

In [79]:
op2

tensor([[ 0.0822, -0.0256,  0.9837,  ...,  0.9976, -0.1429,  0.9998],
        [ 0.0511, -0.0417,  0.9976,  ...,  0.9997, -0.2264,  0.9999],
        [ 0.0466, -0.0839,  0.9969,  ...,  0.9993, -0.1022,  0.9997],
        ...,
        [ 0.0690, -0.0936,  0.9893,  ...,  0.9979, -0.3027,  0.9998],
        [ 0.0745, -0.0237,  0.9895,  ...,  0.9984, -0.2224,  0.9998],
        [ 0.0382, -0.0960,  0.9923,  ...,  0.9986, -0.1203,  0.9998]],
       grad_fn=<TanhBackward>)

In [81]:
len(op2)

16

In [ ]:
# if output_hidden_states=True

In [46]:
len(op4)

12

In [47]:
op4[0].shape #(attention for each layer (batch_size, num_heads, sequence_length, sequence_length))

torch.Size([16, 12, 512, 512])

#### little bit about the BERT model outputs

In [ ]:
# BERT (the base model without any heads on top) outputs 2 things: last_hidden_state and pooler_output.

# last_hidden_state contains the hidden representations for each token in each sequence of the batch.
# size is (batch_size, seq_len, hidden_size)
# pooler_output contains a "representation" of each sequence in the batch, and is of size (batch_size, hidden_size). 
# What it basically does is take the hidden representation of the [CLS] token of each sequence in the batch 
# (which is a vector of size hidden_size), and then run that through the BertPooler nn.Module. 
# This consists of a linear layer followed by a Tanh activation function. 
# The weights of this linear layer are already pretrained on the next sentence prediction task. 

In [ ]:
# alternate hyphothesis (https://github.com/huggingface/transformers/issues/1328)

In [ ]:
# In the paper, they state the following about the pooler output: "This output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence."
# So it's actually better to use outputs[0], which are the hidden representations of all tokens, and take the average. 
# But what actually also works well in practice is just using the hidden representation of the [CLS] token. 
# The reason this [CLS] token is introduced is because it can be used for classification tasks. 
# You can see the hidden representation of the [CLS] token as a representation of the whole sequence (sentence).
# Since outputs[0] is of size (batch_size, seq_len, hidden_size), and 
# we only want the vector of the [CLS] token, we can obtain it by typing outputs[0][:, 0, :]. 
# You can then apply a dropout layer,
# followed by a linear layer on top of that to get 2 outputs (in case you are doing binary text classification).


# example: 
# hidden_states = outputs[0] # here outputs[1] is the 
# pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
# pooled_output = pooled_output[:, 0, :]
# pooled_output = self.dropout(pooled_output)
# # classifier of course has to be 4 * hidden_dim, because we concat 4 layers
# logits = self.classifier(pooled_output)

In [ ]:
# so that's why in the forward pass, we use outputs[1], meaning the output of the pooler layer

## Captum viz

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import sklearn
import sklearn.ensemble
import sklearn.metrics
from scipy.special import softmax

from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn as nn


from captum.attr import IntegratedGradients, LayerIntegratedGradients, InputXGradient
from captum.attr import InterpretableEmbeddingBase, TokenReferenceBase
from captum.attr import visualization
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [2]:
# sens for ke approach

In [ ]:
# 1st patient

In [3]:
# sentence = 'discharge diagnoses acute on chronic hemic respiratory failure and severe sepsis secondary to severe exacerbation copd caused by metapneumovirus and presumptive bacterial cap acute delirium acute pulmonary edema due to acute on chronic combined systolic and diastolic chf acute kidney injury troponinemia anxiety with depression gerd benign essential hypertension coronary artery diseasediastolic dysfunctionischemic cardiomyopathypvd hypernatremia halemia discharge disposition discharge to sh kelsey swing bed followup recommendations patient was recommended to followup with debra house fnp in days patient should followup with doctor mander within month she may benefit from repeat echo vs stress test given decreased ef noted on inpatient echo would wait until her acute respiratory issues have resolved future appointments date time provider department center pm john cantor mdpulgnvl shmg am debra house fnp dnp fmlkv shuh appointments needing to be scheduled contact information for followup spectrum health kelsey hospital ip washington ave lakeview mi phone next steps follow up instructions discharging today to sar instructions for after discharge adult diet complete by as directed call contact emergency services right away for any heart attack warning signs trouble breathing chest discomfort or discomfort in other areas upper body other common symptoms include cold sweats nausea or lightheadedness complete by as directed call contact emergency services right away for any stroke warning signs sudden numbness or weakness face arm or leg especiall one side confusion trouble speaking trouble seeing trouble walking or if you have really bad headache with no known cause complete by as directed call doctor for persistent nausea or vomiting complete by as directed call doctor for redness tenderness or signs infection at iv or procedure sites complete by as directed call doctor for severe uncontrolled pain complete by as directed call doctor for temperature greater than complete by as directed call doctor if red blood found in stool or vomit complete by as directed do not smoke or use tobacco products complete by as directed early and progressive mobility complete by as directed occupational therapy evaluate and treat at receiving facility complete by as directed physical therapy evaluate and treat at receiving facility complete by as directed read new medication instructions complete by as directed resume normal activity as tolerated complete by as directed resume normal diet complete by as directed details hospital stay presenting problem copd exacerbation hcc j acute exacerbation chronic obstructive pulmonary disease copd hcc j acute exacerbation chronic obstructive pulmonary disease copd hcc j hospital course charlene is an yearold female with histor oxygen and steroiddependent copd coronary artery disease ischemic cardiomyopathy peripheral arterial disease depression with anxiety gerd and hyperlipidemia who presented to emergency department on with increased shortness breath and abdominal pain she normally wears oxygen at baseline and was smoking approximately half pack cigarettes per day prior to presentation she presented to kelsey emergency department and was in obvious respiratory distress and was immediately placed on bipap therapy chest xray showed no acute abnormality labs showed creatinine normal wbc count troponin bnp and lactic acid she was transferred to united hospital on bipap for suspected acute copd exacerbation and clinical pneumonia patient was admitted to intensive care unit at united hospital she was continued on bipap therapy she was placed on mucinex iv solumedoctorl rocephin azithromycin scheduled nebs and iv fluids she had an acute worsening in her shortness breath on morning chest xray was repeated at that time and suggested acute pulmonary edema she was placed on lovenox subq bid and her iv fluids were stopped she was given dose iv lasix her bnp remained quite elevated over next several days and her lasix was increased to twice daily she had good diuresis while on lasix she developed very extensive bruising while on lovenox so this was stopped on an echocardiogram was performed and revealed ejection fraction with moderately hinetic lateral wall and inferior lateral wall there was suggestion diastolic dysfunction mild tricuspid regurgitation and rvsp mm hg patient had very slow improvement over course hospitalization she was gradually able to be weaned back to her baseline oxygen and was transferred out intensive care unit she had intermittent episodes confusion consistent with acute delirium throughout remainder hospitalization she was seen by physical and occupational therapy both whom recommended subacute rehab at discharge she was felt stable for discharge to sh kelsey swing bed on she was on her baseline oxygen for greater than hours prior to discharge she will complete prednisone taper upon discharge she had completed full course antibiotics prior to discharge so she will not be antibiotics at time discharge she may follow up with debra house fnp in days would recommend followup with cardiology within next month given her doctorp in ef down to with wall motion abnormalities given her endstage copd in setting acute illness did not think she would tolerate stress test or invasive cardiac intervention during this hospitalization but this could be revisited when she recovers from this hospitalization consults recommendation none inpatient procedures none pertinent labs and studies lab results component value date wbc rbc hgb hct mcv platelet lab results component value date glucose sodium potassium chloride hco aniongap bun creatinine egfr echocardiogram interpretation summary left ventricular ejection fraction is lv ejection fraction is mildly decreased there is no thrombus lateral wall is moderately hinetic inferior lateral wall is moderately hinetic diastolic dysfunction is present based on medial evelocities was msec lv systolic dysfunction is also present there is moderate mitral valve thickening there is no mitral valve stenosis there is mild tricuspid regurgitation estimated right ventricular systolic pressure is mmhg estimated pulmonary artery systolic pressure is mmhg there is moderate pulmonary hypertension todays study was compared to one performed on lvef was on prior report interpreted by doctor paul singh cardiologist discharge medications acetaminophen tylenol tablet take tablets by mouth every hours as needed albuterol proair hfa base mcgact inhaler inhale puffs every four hours as needed for wheezing amlodipine norvasc tablet take by mouth daily start on asas needed chewable tablet chew tablet daily atorvastatin lipitor tablet take one tablet by mouth daily budesonide pulmicort mgml nebulizer solution take by nebulization daily as needed calcium carbonate calcium po take by mouth times daily citalopram celexa tablet take tablet by mouth daily furosemide lasix tablet take one tablet by mouth daily for diuretic gabapentin neurontin capsule take one capsule by mouth nightly guaifenesin mucinex hr tablet take tablet by mouth times daily ipratropiumalbuterol duoneb mgml nebulizer solution take ml by nebulization times daily loratadine allergy relief tablet take tab by mouth nightly losartan cozaar tablet take one tablet by mouth daily magnesium tabs take by mouth daily metoprolol tartrate lopressor tablet take tablet by mouth times daily prednisone deltasone tablet take tabs daily for days then take tabs daily for days then take tab daily for days then decrease back to daily which is your chronic maintenance dose prednisone deltasone tablet take one tablet by mouth daily symbicort mcgact inhaler inhale puffs two times day trazodone desyrel tablet take one tablet by mouth nightly spent minutes performing this hospital discharge'

In [4]:
# sentence = 'great lakes caring kenmoor ave se ste grand rapids mi great lakes caring home heath services will be contacting you within hours your discharge to schedule your home physical and occupational therapy if you do not hear from them within days your discharge or if you have any questions you can contact them at montcalm county commission on aging state po box stanton mi you can contact montcalm county commission on aging at to inquire about programs regarding other inhome services as well as day programs'

In [5]:
# sentence = 'acute exacerbation chronic obstructive pulmonary disease copd hcc severe sepsis copd with acute exacerbation hcc metapneumovirus positive mild cognitive impairment hia aaa abdominal aortic aneurysm hcc pvd peripheral vascular disease hcc tobacco abuse discharge medications start on amlodipine norvasc tablet take tablet by mouth daily start on potassium chloride sa kdur klorcon meq controlled release tablet take tablet by mouth daily for potassium start on prednisone deltasone tablet take tablet by mouth daily active issues requiring followup discharge disposition home with services future appointments date time provider department center pm john cantor mdpulgnvl shmg am debra house fnp dnp fmlkv shuh appointments needing to be scheduled instructions for after discharge great lakes caring kenmoor ave se ste grand rapids mi great lakes caring home heath services will be contacting you within hours your discharge to schedule your home physical and occupational therapy if you do not hear from them within days your discharge or if you have any questions you can contact them at montcalm county commission on aging state po box stanton mi you can contact montcalm county commission on aging at to inquire about programs regarding other inhome services as well as day programs details stay presenting problem post viral debility r essential hypertension assessment & plan bp lower than goal reduce norvasc and monitor physical deconditioning assessment & plan ambulation via walker with assistance followed by ptot support and encourage therapy participation unspecified severe proteincalorie malnutrition hcc assessment & plan dc statin as nothree times daycate ane insomnia unspecified type assessment & plan take trazadone for this home med continue anxiety assessment & plan patient denies anxious feeling at this time currently has xanax for anxiety continue with support care osteoporosis assessment & plan chronic discomfort pain being managed with tylenol continue to monitor chf congestive heart failure hcc assessment & plan no cardiac symptoms currentl lasix cozaar and lopressor recommendation to follow up with cardiology in one month continue to monitor pt with poor functional status not good cardiac intervention candidate copd very severe hcc assessment & plan patient currentl lmin which is home level currentl albuterol symbicort duoneb prednisone taper continue to monitor pt would be hospice appropriate if this is pt family goal at this time pt ok with repeat hospitalizations and intubation though pt may not fully understand what these decisions mean bp pulse temp oral resp ht wt kg spo bmi kgm fio physical exam constitutional no distress elderlygen dependant female examined in assigned swing bed room hent mouththroat oropharynx is clear and moist eyes pupils are equal round and reactive to light neck no jvd present no tracheal deviation present cardiovascular normal rate regular rhythm and normal heart sounds pulmonarychest effort normal diminished in lower lobes bilaterally lmin nc baseline abdominal soft bowel sounds are normal she exhibits no distension there is no tenderness there is no guarding musculoskeletal limited rom moves all extremities spontaneously neurological she is alert skin skin is warm and doctor she is not diaphoretic psychiatric pleasant cooperative'

In [2]:
# final sentence 
sentence = 'discharge diagnoses acute blood loss anemia secondary to acute upper gi bleed proximal pancreatic mass with suspected liver mets pathology consistent with pancreatic adenocarcinoma severe copd with chronic hemic respiratory failure mild to moderate cognitive impairment depression with anxiety hyperlipidemia chronic combined systolic and diastolic chf benign essential hypertension coronary artery disease stage ckd discharge disposition discharge to home followup recommendations patient was recommended to followup with oncology within weeks patient was recommended to followup with pcp in days future appointments date time provider department center am debra house fnp dnp fmlkv shuh appointments needing to be scheduled contact information for followup cancer hematology centers western michigan pc kenmoor ave se ste grand rapids mi phone next steps follow up instructions they will call you with date and time for your appointment melissa wilson fnp specialty nurse practitionerfamily nurse practitioner relationship pcp general spectrum health washington ave lakeview mi phone next steps schedule an appointment as soon as possible for visit in weeks instructions for after discharge call doctor for persistent nausea or vomiting complete by as directed call doctor for redness tenderness or signs infection at iv or procedure sites complete by as directed call doctor for severe uncontrolled pain complete by as directed call doctor for temperature greater than complete by as directed call doctor if red blood found in stool or vomit complete by as directed do not smoke or use tobacco products complete by as directed low cholesterol low fat diet complete by as directed read new medication instructions complete by as directed ref to hematologyoncology spectrum health complete by as directed resume normal activity as tolerated complete by as directed details hospital stay hospital course charlene is an yearold female with histor cad dementia oxygendependent copd chronic combined systolic and diastolic chf stage ckd hyperlipidemia and depression with anxiety who presnted to ed with abdominal pain nausea and coffeeground emesis initial lactic acid normal liver function testing initial hemoglobin was gdl she was hensive in ed she was given fluids and units prbcs ct abdomenpelvis was done and showed proximal pancreas mass causing extrinsic compression celiac trunk and its branches as well as splenic vein and superior mesenteric vein there was evidence metastatic disease in liver general surgery evaluated patient in ed and did not feel that urgent surgical intervention was necessary she was admitted to hospitalist for further management upon admission patient was placed on protonix doctorp she was made npo gi was consulted she underwent egd with endoscopic ultrasound on biopsies were obtained pathology returned positive for adenocarcinoma likely pancreatic primary following this procedure patient was started on clear liquid diet and was advanced as tolerated her hemoglobin did doctorp to on and she was given an additional unit packed red blood cells her hemoglobin has remained stable since that time she has not had any further evidence bleeding while hospitalized did let family now pathology results and have recommended outpatient followup with oncology for any further treatment options discussed with family that thought treatment was unlikely and that comfort care would be recommended family at bedside seemed to understand patient has underlying dementia and am confident she did not grasp this diagnosis she was felt stable for discharge home on outpatient referral to oncology was placed at discharge she may followup with her pcp in days asas neededwill be held at discharge and she will be discharged on high dose ppi protonix po bid consults recommendation gastroenterology inpatient procedures egd with eus pertinent labs and studies lab results component value date wbc rbc hgb hct mcv platelet lab results component value date glucose sodium potassium chloride hco aniongap bun creatinine egfr ct abdomenpelvis with contrast metastatic lesions within liver abnormal proximal pancreas mass causing extrinsic compression celiac trunk and its branches as well as splenic vein and superior mesenteric vein where they meet to become portal vein gastritis suspected invasion distal stomach or proximal duodenum by pancreatic lesion cannot be ruled out no sign active arterial bleeding but this does not rule out slower venous bleeding interpreted by doctor robert deguzman radiologist discharge medications acetaminophen tylenol tablet take tablets by mouth every hours as needed albuterol hfa proventil hfa ventolin hfa proair hfa base mcgact inhaler take puffs by inhalation every hours as needed for wheezing amlodipine norvasc tablet take by mouth daily atorvastatin lipitor tablet take by mouth nightly busas needed buspar tablet take by mouth nightly calcium carbonate calcium po take by mouth times daily cholecalciferol vitamin d units caps take units by mouth daily citalopram celexa tablet take tablet by mouth daily erythromycin romycin ophthalmic ointment apply application to both eyes daily as needed doctorinage apply to affected eyes nightly for antibiotic ferrous sulfate doctored slow iron fe tbcr take tablet by mouth daily fluticasoneumeclidinvilant trelegy ellipta mcginh aepb take act by inhalation daily furosemide lasix tablet take by mouth daily gabapentin neurontin capsule take by mouth nightly ipratropiumalbuterol duoneb mgml nebulizer solution take ml by nebulization every hours as needed for wheezing loratadine allergy relief tablet take tab by mouth nightly magnesium tabs take by mouth daily metoprolol tartrate lopressor tablet take tablet by mouth times daily multiple vitaminsminerals multivitamin adults po take tab by mouth daily omeprazole prilosec capsule take cap by mouth times daily minutes before breakfast and dinner oxymetazoline afrin nasal spray administer sprays into affected nostrils times daily as needed pantoprazole protonix tablet take tablet by mouth times daily before meals for days polyethylene gl miralax bulk powder take by mouth daily potassium chloride sa kdur klorcon meq controlled release tablet take tablet by mouth daily for potassium probiotic product probiotic daily po take by mouth daily respiratory therapy supplies oxygen therapy devi liter flow l route nasal canula frequency continuous diagnosis copd j please supply patient with portable oxygen concentrator and all supplies patient taking differently liters liter flow l route nasal canula frequency continuous diagnosis copd j please supply patient with portable oxygen concentrator and all supplies spaceraeroholding chambers devi each by does not apply route as needed trazodone desyrel tablet take by mouth nightly vitamin complex po take by mouth daily spent minutes performing this hospital discharge'

In [ ]:
# 2nd patient

In [2]:
# sentence = 'pediatric hematologcology discharge summary name makay james vanoort mrn primary care physician at discharge julia davey md hematologistoncologist david dickens md brief overview makay james vanoort is male with histor suspected alps who is admitted for fever anemia and hyperbilirubinemia problem list as reviewed pm by candie ritsema pnpac hospital principalhemolytic anemia hcc anemia nausea arthralgia fever common variable immunodeficiency with predominant immunoregulatory tcell disorders hcc nonhospital asthma allergic rhinitis cause unspecified dairy product intolerance enrolled in improve care now reactive depression primary insomnia intractable vomiting with nausea unspecified vomiting type thrombocenia hcc jaundice hyperbilirubinemia cyclic vomiting syndoctorme early satiety gad generalized anxiety disorder marijuana use as neededchild relationship problem school problem severe single current episode major depressive disorder hcc sleeping difficulties evans syndoctorme hcc alps autoimmune lymphoproliferative syndoctorme hcc weight loss somatic symptom disorder persistent severe with predominant pain panic disorder without agoraphobia adjustment disorder with mixed anxiety and depressed mood discharge disposition as needed and patient given instructions on when to call with signs infection including temperature greater than degrees fahrenheit signs anemia andor signs bleeding outpatient followup future appointments date time provider department center am brittany barber garcia phd psy shmg am provider shdv hem dvcanchem shdv am app shdv hem dvcanchem shdv am brittany barber garcia phd psy shmg am brittany barber garcia phd psy shmg am nicholas hartog mdhdvchallimm shmg details hospital stay presenting problem fever r hemolytic anemia hcc d makay is year old with suspected alps and hemolytic anemia on immunosupressive therapy with prednisone sirolimus and vincristine he most recently had labs on and his hemoglobin was decreased to and his bilirubin he was feeling well and he was to continue on his daily prednisone and sirolimus makay fell asleep and missed his prednisone and sirolimus on evening he then had fever during night and came to er on he denied any respiratory symptoms he continues to have myalgias but no other new symptoms he does have some chronic nausea that he stated was no worse than his baseline he was tachycardic with low blood pressures so was given ns bolus his blood pressures did normalize prior to being admitted to floor blood & urine cultures were obtained and he was given iv cefepime and an oral dose prednisone he was admitted to th floor with fever and anemia hospital course makay was transitioned to daily empiric iv rocephin as he was not neutropenic his remained afebrile and blood culture negative throughout rest admission he received therapy for his immunerelated disease process during admission continuing on daily prednisone daily sirolimus and dose subcutaneous velcade on for symptoms nausea he received intermittent doses zofran zyprexa benadoctorl andor marinol for arthralgias he received as needed ibuprofen or tramadol due to therapy related immunesuppression makay received pjp prophylaxis with inhaled pentamidine on and was started on fungal prophylaxis with oral fluconazole due to interactions with fluconazole sirolimus was decreased by and he now receives daily sirolimus trough will be checked at his next outpatient visit on makay was feeling better and ready for discharge to home he was instructed to continue daily prednisone and sirolimus and will return to clinic next week for followup chemotherapy administered if applicable facilityadministered medications as medication dose route frequency completed bortezomib velcade mgml subcutaneous injection mgm treatment plan actual subcutaneous once completed prednisone deltasone tablet oral once completed prednisone deltasone tablet oral once discharge physical exam bp pulse temp oral resp ht wt kg spo bmi kgm physical exam constitutional he is oriented to person place and time he appears welldeveloped and wellnourished hent head normocephalic mouththroat oropharynx is clear and moist eyes conjunctivae and eom are normal mild scleral icterus cardiovascular normal rate normal heart sounds and intact distal pulses pulmonarychest effort normal and breath sounds normal abdominal soft musculoskeletal normal range motion neurological he is alert and oriented to person place and time skin skin is warm and doctor psychiatric he has normal mood and affect pertinent labs and studies recent results from past hours cbc and manual differential status abnormal result value ref range status white blood cell ul final red blood cell ul final hemoglobin gdl final hematocrit final mean cell volume fl final mean cell hemoglobin pg final mean cell hemoglobin concentration gdl final red cell diameter width final platelet ul final mean platelet volume fl final segmented neutrophils manual final neutrophil absolute count ul final lymphocytes manual final monocytes manual final eosinophil manual final basophils manual final rbc morphology see below final anisocis few final polychromasia few final platelet estimate normal final comprehensive metabolic panel status abnormal result value ref range status sodium level mmoll final potassium level mmoll final chloride mmoll final hco mmoll final anion gap mmoll final glucose level mgdl final blood urea nitrogen mgdl final creatinine mgdl final cg ecrcl mlmin m final modified schwartz egfr mlmin m final calcium level total mgdl final protein total gdl final albumin level gdl final bilirubin total mgdl final alkaline phosphatase iul final alanine aminotransferase iul final aspartate aminotransferase iul final hemolysis final reticulocytes status abnormal result value ref range status retic automated red cell count µl final reticulocyte automated final absolute reticulocyte count µl final discharge medications albuterol proventil ventolin mcgact inhaler take puffs by inhalation every hours as needed doctornabinol marinol capsule take capsule by mouth times daily as needed limiting use to once day as refill not allowed by insurance start on fluconazole diflucan tablet take tablets by mouth one time each day at same time for days fluconazole diflucan tablet take tablets by mouth daily for days olanzapine zydis zyprexa disintegrating tablet take by mouth nightly ondansetron zofranodt disintegrating tablet take tablet orally every hours as needed for nausea patient taking differently take by mouth every hours as needed for nausea or vomiting pentamidine solr in dextrose soln ml infuse into venous catheter every month prednisone deltasone tablet take tablets by mouth daily patient taking differently take by mouth daily tablets ranitidine zantac tablet take tab by mouth nightly sirolimus rapamune tablet take tablet by mouth every morning for days tablets tramadol ultram tablet take by mouth every hours as needed for pain spent minutes performing this hospital discharge lisa hovingh pnpac personally examined patient and agree with physical findings as documented have reviewed history physical examination and laboratory findings and agree with documentation plan was discussed and developed with nurse practitioner makay was well appearing on discharge physical exam findings included acneiform rash lungs were clear he was sent home on his prednisone daily and he will return for his next velcade on sharon smith md'

In [2]:
# sentence = 'adoctornal insufficiency hcc secondary to chronic steroid therapy yes fever yes arthralgia yes adjustment disorder with mixed anxiety and depressed mood yes common variable immunodeficiency with predominant immunoregulatory tcell disorders hcc yes evans syndoctorme hcc yes nausea yes reactive depression yes asthma yes resolved hospital problems diagnosis date noted date resolved poa no resolved problems to display discharge disposition home will get local labs tomorrow am in grand haven active issues requiring followup blood culture from ngtd final results pending outpatient followup future appointments date time provider department center pm julia davey mdpeds gh shmg am app shdv hem dvcanchem shdv am brittany barber garcia phd psy shmg pm bradd hemker mdhdpc shrh pm brittany barber garcia phd psy shmg pm brittany barber garcia phd psy shmg pm brittany barber garcia phd psy shmg test results pending at discharge if blank below none details hospital stay presenting problem evans syndoctorme hcc d febrile illness acute r histor neutropenia z sepsis due to unspecified organism hcc a acute non intractable tensiontype headache g sepsis hcc a evans syndoctorme hcc d hospital course picu course from to makay was initially admitted to picu for shock due to holemia versus sepsis he was pan cultured and started on vancomycin and cefepime in emergency department but continued to be hensive and tachycardic despite appropriate fluid resuscitation he was started on dopamine doctorp in picu but tolerated weaning starting on night admission due to maps remaining above he continued to appear better over course his day in picu his reported headache were improving and he was able to keep food down although he did require as needed zofran throughout day since starting on antibiotics he has not spiked any fevers and all cultures no growth to date there was no signs clinically relevant coagulopath exam no new bleeding or bruising since admission prior to transfer he was comfortable on room air and starting to tolerate solid foods makay was transferred out picu on he was initially placed on stress dose steroids upon admission due to his holemia and long term prednisone therapy his sirolimus was held at admission and he was due to receive his velcade on but that dose was deferred to when he was recovered and stable he remained afebrile and on his stress dose steroids were stopped he then resumed his previous dose prednisone on makay had negative blood cultures for hours his viral panal was positive for rhinoenterovirus and he had been afebrile since his iv antibiotics were narrowed from vancomycin and cefepime to cefepime as single agent on on his cefepime was discontinued his hemglobin had decreased to on so his prednisone dose was increased to bid his sirolimus was restarted at daily and he was given his velcade his platelets were also on mild trend downward during his admission at discharge makay and his mother were aware importance to get labs on morning he requires close follow up given his recent hematologic trends and medication changes makay continued on his other daily medications including his seroquel marinol zantac methadone zofran and rad medications his neurontin was increased to bid during his admission consultsrecommendation picu chemotherapy administered if applicable facilityadministered medications as medication dose route frequency completed bortezomib velcade mgml subcutaneous injection subcutaneous once prednisone deltasone tablet oral bid completed prednisone deltasone tablet oral once sirolimus rapamune tablet oral daily discharge physical exam bp pulse temp oral resp wt kg lb oz spo bmi kgm physical exam see progress note from today pertinent labs and studies recent results from past hours comprehensive metabolic panel cmp status abnormal result value ref range status sodium level mmoll final potassium level mmoll final chloride mmoll final hco mmoll final anion gap mmoll final glucose level mgdl final blood urea nitrogen mgdl final creatinine mgdl final cg ecrcl mlmin m final modified schwartz egfr mlmin m final calcium level total mgdl final protein total gdl final albumin level gdl final bilirubin total mgdl final alkaline phosphatase iul final alanine aminotransferase iul final aspartate aminotransferase iul final complete blood count with manual differential status abnormal result value ref range status white blood cell ul final red blood cell ul final hemoglobin gdl final hematocrit final mean cell volume fl final mean cell hemoglobin pg final mean cell hemoglobin concentration gdl final red cell diameter width final platelet ul final mean platelet volume fl final neutrophil absolute count ul final differential manual blood status abnormal result value ref range status segmented neutrophils manual final bands manual final lymphocytes manual final monocytes manual final eosinophil manual final basophils manual final metamyelocyte manual final myelocyte manual final rbc morphology see below final platelet estimate decreased final anisocis few final immature platelet fraction ipf status normal result value ref range status immature platelet fraction final electrocardiogram complete status none narrative ventricular rate bpm atrial rate bpm pr interval ms qrs duration ms qt interval ms calculationbezet ms calculated axis degrees calculated axis degrees calculated axis degrees diagnosis sinus rhythm with marked sinus arrhythmiawhen compared with ecg dec unconfirmednonspecific wave abnormality now evident in anterior leadsconfirmed by alkhatib yasser on pm discharge medications albuterol proventil ventolin proair base mcgact inhaler take puffs by inhalation every hours as needed for wheezing doctornabinol marinol capsule take capsule by mouth times daily as needed doctornabinol marinol capsule take cap by mouth times daily for days fluticasone furoatevilanterol breo ellipta inh take puff by inhalation at bedtime gabapentin neurontin capsule take capsules by mouth times daily for days methadone dolophine tablet take tablets by mouth times daily for days methadone dolophine tablet take by mouth times daily ondansetron zofranodt disintegrating tablet take tablet orally every hours as needed for nausea polyethylene gl miralax packet take by mouth daily as needed prednisone deltasone tablet take tablet by mouth times daily quetiapine seroquel tablet take by mouth at bedtime ranitidine zantac tablet take tablet by mouth nightly for days sirolimus rapamune tablet take tablets by mouth every morning spent minutes performing this hospital discharge candie ritsema np personally examined patient and agree with physical findings as documented see progress note have reviewed history physical examination and laboratory findings and agree with documentation plan was discussed and developed with nurse practitioner beth kurt md'

In [ ]:
# end for ke's apporach

In [22]:
# device = 'cpu'

In [23]:
# big
# missclass_path = '../src/pubmedbert_1ep_3e-05lr_1616bt_nosampler_noextparams_yessch_notlast_mixedapproach__can.pt_misclass_full.csv'

In [4]:
# df = pd.read_csv(missclass_path)

In [22]:
# df[df['Y_TRUE'] == 0]

In [5]:
# df.head(20)

In [ ]:
# positive, missclass as negative, ? let's see

In [38]:
# sentence = 'medicine iodine ct contrast attending chief complaint weakness major surgical or invasive procedure intubation mechanical ventilation extubation incision and doctorinage right axillary abscess placement central venous catheter the patient is yo with histor metastatic melanoma with spine brain and adoctornal mets who has had left sided weakness for months and has been developing right sided weakness over the past week thought to be related to known c mass presented to the ed today due to inability to perform adls the patient was last seen by his oncologist doctor on at that time he was started on tpch chemotherapy regimen day additionally the patient was recently seen by his pcp for the left sided weakness and his dexamethasone regimen was increased to qh with minimal improvement he denies any loss bowelbladder continence no other neurological sx denies feverchills has had significant diffulty maintaining appropriate glucose levels since starting on steroids note the patient has also developed right axillary abscess over the past week started on keflex on by np at oncologists office this has been increasing in size began spontaneuosly doctorining today when palpated by paramedics en route to the ed the patients fs was noted to be and he received humalog at that time in the ed the patients intial vitals were hr bp rr and spo ra laboratory studies revealed na and given amp calcium gluc and an ecg was performed showing without peaked ts cxr showed no acute intrathoracic process received home insulin and became slightly hlycemic had bp systolic and received l ivf with good response got vanc x for his abcess iv in ed infiltrated and its unclear how much the above the patient received on the floor the patient is mentating well somewhat hensive with bas neededing from systolic otherwise no acute complaints ros as per hpi otherwise denies any cp palp sob nvd feverchills changes in bowelbladder habits weight loss ha or vision changes oncologic history he biops mm thick primary melanoma without ulceration from his right mid back in he sentinel lymph node biopsy without evidence melanoma in his right axillary nodes he was enrolled in ecog protocol randomized to the observation arm he was well until when cxr revealed multiple new pulmonary nodules with ct confirmation he was referred to interventional pulmonary doctor for bronchoscopic biopsy brain mri on revealed multiple brain mets and petct on revealed widespread metastatic disease including worrisome cervical spine lesion on he cspine mri confirming intramedullary metastasis at cc he was seen urgently by doctor orthopedics and felt not to be surgical candidate he bronchoscopy and left adoctornal biops with the left subcarinal ln showing no malignant cells and the left adoctornal bx cw metastatic melanoma he completed day course cspine radiation at hospital on he began ipilimumab on the brain metastasis trial on and did very well he had slow growth in the lll lung lesion and or bronchoscop with for bronchoscopy biopsy and debridement tumor pathology confirmed metastatic melanoma he vats lll wedge resection by doctor on pathology confirmed melanoma in the lung lesion but margins were clear and lymph nodes were negative for tumor cspine mri was performed in to evaluate lue numbness noted intermittently since rigid bronchoscop with slight increase in size cc mass felt rt inflammation from ipilimumab he subsequently developed progressive left shoulder arm and neck pain and increased ue numbness prompting er evaluation on cspine mri revealed slight increase in the cc mass and increased edema and he was started on prednisone his brain mri and torso ct were stable without disease progression tumor is braf wild type he was removed from the ipilimumab brain metastasis clinical trial due to symptoms and interval increase in the cc mass although ipilimumab related inflammation rather than true disease progression was felt to be possible attempts to lower steroid dosing were unsuccessful started on temodar diabetes diagnosed in hypertension hypercholesterolemia vasectomy he is retired from the insurance industry and also taught computer at private middle school on pack year smoker quit in doctornks per month no doctorgs had an uncle who died metastatic melanoma his grandmother had breast cancer and his brother died an mi in his sleep at age his sister also has cardiac problems physical exam admission physcial exam vitals l general appears well and in nad joking in bed heent perrla eomi anicteric mm doctor op clear cv rrr s and s no mrg lung ctab no withrr abdomen soft ntnd bsx extremity two areas erythema on the right anterior chest superior lesion is fluctuant and doctorining size is approx cm neuro awake alert and oriented strength in lue and lle in rue and rle some numbmness to palpation diffuseler extremities discharge physical exam vs bp hr rr spo on ra general nad aox heent doctor mm op clear no oral ulcerations or exudate neck no jvd cv rrr no mrg lungs clear to auscultation anteriorly no crackles or wheezes abd bs ntnd soft ext wwp soft pitting edema bilaterally no clubbing or cyanosis right axillary wound with bandage cdi neuro strength in the uele on the right strength in the uele on the left pertinent results admission labs pm blood wbc rbc hgb hct mcv mch mchc rdw plt ct pm blood neuts bands lymphs monos eos baso atyps metas myelos pm blood pt ptt inrpt am blood fibrino pm blood glucose urean creat na k cl hco angap pm blood alkphos totbili pm blood alkphos totbili am blood ldldh ckcpk pm blood lipase pm blood albumin am blood calcium phos pm blood osmolal am blood typemix po pco ph caltco base xs pm blood glucose na k cl calhco am blood lactate discharge labs am blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood neuts bands lymphs monos eos baso atyps metas myelos am blood pt ptt inrpt am blood glucose urean creat na k cl hco angap am blood ldldh alkphos totbili am blood albumin calcium phos other relevant labs am blood vanco am blood am blood t free t pm urine hoursrandom creat na k cl pm urine osmolal microbiology pm blood culture source lineleft sc blood culture routine final pseudomonas aeruginosa final sensitivities pseudomonas aeruginosa cefepime ceftazidime ciprofloxacin gentamicin meropenem piperacillintazo tobramycin am swab site right axilla gram stain final per x field polphonuclear leukocytes per x field gram negative rods anaerobic culture final no anaerobes isolated fungal culture preliminary no fungus isolated wound culture final pseudomonas aeruginosa heavy growth enterococcus sp rare growth pseudomonas aeruginosa enterococcus sp ampicillin cefepime ceftazidime ciprofloxacin gentamicin meropenem penicillin piperacillintazo tobramycin vancomycin imaging studies chest pa & lat at pm findings the lungs are clear without consolidation or edema the mediastinum is unremarkable the cardiac silhouette is within normal limits for size no effusion or pneumothorax is noted degenerative changes are seen throughout the thoracic spine impression no acute pulmonary process mr at pm findings the intramedullary heterogeneously enhancing lesion at cc level is stable in size and morphology currently measuring craniocaudal anteroposterior coronal mm likewise the associated cord edema is unchanged notably extending from c to c level at the level the lesion the spinal cord is expanded and there is unchanged severe spinal canal stenosis with loss csf signal stenosis at this level is exacerbated by unchanged central disc protrusion there is no interval change with regard to multilevel multifactorial degenerative changes most notably with moderate neural foraminal stenosis on the left at cc level and multilevel disc bulges at cc cc and cc no evidence new metastatic lesions the visualized paraspinal soft tissues are unremarkable impression stable appearance cervical intramedullary metastatic lesion with extensive cord edema cta abdomen pelvis at pm ct abdomen there is moderate subcutaneous emphysema along the right anterior chest wall presumably postprocedural the lung bases are clear there is no pleural effusion the heart is normal in size without pericardial effusion there is multivessel coronary arterial disease intraabdominal visceral evaluation is highly limited on current examination due to significant streak artifacts however allowing for such there is no focal liver lesion the gallbladder spleen and adoctornal glands are unremarkable bilateral kidneys enhance symmetrically without htornephrosis or htorureter an enteric tube is in place with tip coiled within the stomach small and large bowel loops are normal in caliber without wall thickening or obstruction there is no evidence active intestinal hemorrhage on current examination extensive colonic diverticulosis is present without diverticulitis hepatic arterial anatomy is conventional the celiac trunk sma and renal arteries are patent is also patent moderate atherosclerotic calcifications are present in the infrarenal aorta ct pelvis the bladder is partially collapsed containing nondependent air likely related to foley catheter placement the rectum appears within normal limits small bilateral fatcontaining inguinal hernias are present there is no pelvic sidewall or inguinal lymphadenopathy by size criteria subcentimeter external iliac and inguinal lymph nodes are stable cm mass in the right upper posterior chest wall soft tissues a is similar as compared to bone window no focal concerning lesion impression no ct evidence active lower gi bleed colonic diverticulosis likely postprocedural right anterior chest wall subcutaneous emphysema correlation with any recent histor instrumentation is recommended stable right upper posterior chest wall soft tissue mass unchanged subcentimeter iliac and inguinal lymph nodes mr at pm findings cervical vertebrae are normal in height and alignment craniocervical junction appears normal as compared to the previous mri from there has been no interval change in the size and morpholog intramedullary enhancing lesion at cc level again noted is extensive cord edema surrounding the lesion extending from c to c levels multilevel degenerative changes in the cervical spine are unchanged no new metastatic lesions are seen pre and paravertebral soft tissues are unremarkable impression no interval change in cervical intramedullary metastatic lesion with unchanged spinal cord edema bilat lower ext veins at pm findings the common femoral superficial femoral and the popliteal as well as the deep veins the calves on both sides show normal ultrasound appearance compressibility and doppler waveforms conclusion the ultrasound examination was negative for dvt in either lower extremity brief hospital course hensionhyperglycemia first icu course upon admission the initial concern was for sepsis in the setting an axillary abscess patients blood pressures responded well to fluids and he stabilized quickly in the ed patients blood glucose was so likely he was simply volume depleted secondary to osmotic diuresis in his hyperglycemic state he also had numerous electrolyte abnormalities including hatremia and hyperkalemia which were corrected prior to transfer to the floor second icu course documented under gi bleed see below three times dayicu course on hospital day patient developed hension and tachycardia in the setting missing doses unasyn due to loss iv access he was intubated for airway protection transfered back to the icu and fluid resuscitated his antibiotics were broadened to vanczosyn and he responded well hemodynamically he was able to be extubated after less than hours on his three times dayicu course his hension was attributed to sepsis in the setting pseudomonas bacteremia which was thought to be due to infection the right axillary abscess with pseudomonas also isolated from culture the axillary wound axillary abscess the patient developed an axillary abscess week prior to admission which started spontaneously doctorining the da admission the patient had been on keflex since and received dose vancomycin in ed the patients presentation was likely related to patients immunosupression with chronic steroids and chemo the patient was covered with vancomycin and zosyn initially surgery incised and doctorined the abscess infectious disease recommended vancomycin and unasyn after the patients three times dayadmission to the the axillary abscess was noted to have grown pseudomonas and enterococci he was initially restarted on vancomycin and zosyn as susceptibilities returned the patient was treated with vancomycin and ciprofloxacin gi bleeding on hospital day patient was noted to pass brbpr and clots per rectum in setting platelets he was transiently hensive to the s systolic but recovered his pressures to the low s high s he was transferred to the where ng lavage returned clear gastric contents he was transfused units prbc the source the bleeding was presumed to be from diverticular bleed given his neutropenia the patient was not scoped patient had cta the abdomen and pelvis showed no evidence active lower gi bleed but the presence colonic diverticulosis he stabilized and improved with platelet transfusion ddavp ivf and prbc while in the unit on the floor the patient had no further episodes gi bleeding rue weakness most likely due to the patients c vertebral mass and related nerve impingment no other signs suggestive cord compression initially changed dexamethasone to htorcortisone due to hension and electrolyte abnormalities but switched back to patients original steroid dosing once he was stabilized obtained mri cspine and brain which did not show interval progression metastatic disease though the study was technically limited the patient right upper and lower extremity weakness improved through his hospital course while working with pt and ot acute kidney injury prior to hospitalization patient had baseline creatinine on presentation his creatinine was and continued to climb to elevated creatinine was attributed to prerenal status the patients serum creatinine improved with fluids to at the time transfer to the oncology floor his serum creatinine remained stable through the remainder his hospitalization melanoma when the patient initially presented he had been started on new chemotherapy regimen per hematologyoncology recs tpch chemotherapy was held because acute kidney injury the patients chemotherapy was held through the admission the patient did undergo an mri the brain and cervical spine to monitor the patients disease and it did not show interval progression disease diabetes mellitus type patients finger stick blood glucose were checked with meals and prior to bedtime initially the patient refused to be managed by nurses for insulin and wanted to take his home medications he later agreed to follow an insulin sliding scale and was started on insulin glargine as well his blood sugars were difficult to control during his stay due to his continued high dose steroids his evening lantus dose was uptitrated several times but he continued to have elevated fingersticks at the time discharge his fbgs were running in the s he will need close followup from endocrinology after discharge for further adjustment his insulin regimen hypertension given the patient initial presentation hension his home lisionpril was held during his hospitalization and discontinued at discharge hyperlipidemia his home rosouvastatin was continued through the admission thrombocenia patient initially presented with thrombocenia thought to be due to chemotherapy he received platelets early during his admission to prevent gi bleeding with transfusion goal to keep platelets greater than the patients platelet count recovered later in his hospitalization hhdism patients home dose synthroid was continued through much the hospitalization was doctorwn that returned elevated but with normal free t no changes were made to the patients synthroid he will need repeat tfts weeks from discharge as well as endocrine followup elevated ldh his ldh rose since his last admission to the his hematocrit remained stable as has his tbili suggesting that the elevated ldh was not due to hemolysis the patients and were within normal limits previously were trending down in the setting shocked liver secondary to sepsis which does not suggest liver pathology given the patients dexamethasone it was possible that the rise in ldh may represent pcp infection though the patient denies shortness breath and he had been saturating well on room air ldh has also been used as tumor marker for melanoma so it is possible that the rise may present the patients underlying metastatic disease because the patients prolonged dexamethasone course he was started on pcp prophylaxis with bactrim ds hatremia patient was initially admitted to the with hatremia thought to be due to poor oral intake urine lytes were consistent with siadh in the setting his known metastatic melanoma upon the patients second course his sodium improved with stress dose steroids on the floor as his dexamethasone was being weaned his serum sodium was noted to be falling his dexamethasone was increased back to po qh he was briefly treated with fludoctorcortisone but this was discontinued at discharge endocrine consult felt that an adoctornal or thd etiology for his hatremia was unlikely but that he will need slow taper his steroids given his prolonged high dose course his na was fairly stable around at discharge he will need to follow up with endocrinology soon after discharge for management his steroid taper transition care continue vancomycin and ciprofloxacin for total weeks after negative blood culture on consider infectious disease followup for further antibiotic recommendations monitor serum sodium initially every days and then weekly as sodium levels stabilize endocrinology followup within weeks discharge for hhdism diabetes management and steroid taper slow steroid taper over several months per endocrine recs repeat tfts weeks from discharge continue to work with pt and ot at rehab oncology followup as scheduled soon after discharge medications on admission dexamethasone tablet tablets by mouth every six hours take wtih tablet dexamethasone tablet tablets by mouth hours take with tablet hdoctorxyurea take tabs every hrs for times and then tabs every hrs for nine times insulin lispro protam & lispro humalog mix prescribed bher provider unitml suspension up to units with meals insulin lispro humalog prescribed bher provider unitml cartridge cover dose levothine levothroid mcg tablet tablets by mouth once day taken in the morning lisinopril tablet tablets by mouth once day lorazepam tablet tablets by mouth at bedtime as needed for insomnia lormustine tabs capsules once for total dose nystatin unitgram powder apply topically twice day as needed for groin rash omeprazole prescribed bher provider capsule delayed releasee capsules by mouth daily daildansetron hcl tablet tablets by mouth take tab for nausea as needed no substitution oxone tablet tablets by mouth times daily as needed procarbazine matulane capsule capsules by mouth take tabs every hrs take tabs every hrs for times and tabs every hrs for times no substitution rosuvastatin crestor prescribed bher provider tablet tablets by mouth once day testosterone testim gram gel topically daily thioguanine tabloid tablet tablets by mouth tabs every hrs take tabs every hrs for times then tabs every hrs for times no substitution asas needed tablet discharge medications levothine mcg tablet sig one tablet po daily daily ciprofloxacin in dw ml piggyback sig one dose intravenous qh every hours vancomycin recon soln sig recon solns intravenous h every hours docusate sodium capsule sig one capsule po bid times day sulfamethoxazoletrimethoprim tablet sig one tablet po monday wednesday friday dexamethasone tablet sig one tablet po qh every hours oxone tablet sig tablets po qh every hours as needed for pain rosuvastatin tablet sig one tablet po once day acetaminophen tablet sig tablets po qh every hours as needed for fever or pain simethicone tablet chewable sig one tablet chewable po once day times day as needed for gas collagenase clostridium hist unitg ointment sig one appl topical daily daily applied to gluteal wound senna tablet sig one tablet po bid times day as needed for constipation omeprazole capsule delayed releasee sig one capsule delayed releasee po once day lantus unitml solution sig forty six units subcutaneous at bedtime humalog unitml solution sig one injection subcutaneous four times day before meals and at bedtime according to sliding scale discharge disposition extended care facility hospital and islands discharge diagnosis primary diagnosis pseudomonas bacteremia and sepsis right axillary abscess lower gastrointestinal bleeding steroid induced hyperglycemia hatremia secondary diagnosis metastatic melanoma diabetes mellitus type hhdism discharge condition mental status clear and coherent level consciousness alert and interactive activity status out bed with assistance to chair or wheelchair discharge instructions dear mr it was pleasure taking care your during your hospitalization at during this hospitalization you were admitted to the icu for concerns sepsis you were stabilized and then transferred to the oncology floor you then developed bleeding from your gi tract with low blood pressure and had to be transferred back to the icu for stabilization you had three times darse in the icu because pseudomonas infectious the right axilla that spread into your blood you are currently being treated with antibiotics for your infection please take all medications as instructed the following medication changes have been made start ciprofloxacin iv every hours start vancomycin iv every hours start bactrim ds tab on monday wednesday friday start lantus insulin units at bedtime start humalog insulin according to sliding scale changed levothine mcg by mouth daily stop lisinopril by mouth daily you will need close followup with endocrinology for your diabetes hhdism and steroid taas needed you will also need infectious disease followup for management the antibiotics course for your axillary abscess and recent bacteremia you also have followup scheduled with your oncologist next week please keep all appointments as listed below followup instructions department hematologyoncology when tuesday at pm with np twice dayng campus east best parking garage department hematologyoncology when tuesday at pm with doctor twice dayng sc clinical ctr campus east best parking garage department radiology when tuesday at am with radiology mri twice dayng campus east best parking garage md'

In [2]:
# sentence = 'neurology bactrim furosemide cefepime attending chief complaint altered mental status major surgical or invasive procedure egd with injection epineas needed into duodenum for hemostasis hpi yearold gentleman recently diagnosed by biopsy with gliomatosis cerebri grade ii astroca in status post xrt temodar patient was recently admitted at for pcp pneumonia which he developed in the setting increasing symptoms course cb hic respiratory failure requiring intubation transfer to icu during icu course developed hension requiring pressors occurring in the setting hypersensivity reaction to bactrim and one episode hematochezia during anticoagulation for previous pe following transfer to floor ms albeit slowly improved patient was discharged to with plans to complete day course primaquine and clindamycin allergy to bactrim concern for pancreatitis with pentamidine patient was then admitted to on for altered mental status per dc summary pt had done well and his ms had improved since his dc from such that he was conversant with family on the pt was found unresponsive febrile sent to intubated for resp failure abg on while intubated lac cxr revealed patchy air space opacities cw multifocal pna pt underwent bronchoscopy with witnessed aspiration during the procedure and was started on cefepimevancomycinmetronidazole pcp smears were neg but sputum cx grew out extubated h post admission he was also started on fragmin to histor dvt he was transferred from to for continuit his care followed by doctor per dw family nights prior to this admission pt was sitting up conversant he then had slow decompensation over the next day and was brought back into for eval and treatment coronary artery disease status post mi years ago hypertension hyperlipidemia djd in the cspine depression histor dvt shingles recently finished treatment gliomatosis cerebri grade ii astroca the left temporal lobe with extension into the optic chiasm and left optic nerve diagnosed in after presentation with retrobulbar ischemic optic neuritis status post open brain biopsy status post xrt and low dose temodar repeat mri stable status post high dose temodar landfill workerretired pky histor smoking quit yrs agodenies etohdoctorgs noncontributory physical exam pe t bp p r ra gen arousable to pain opens eyes follows simple commands heent pupil reactive fixed at mm white confluent lesion roof mouth small linear mucosal tear upper side lungs bl rhonchi anteriorly cv regular ss no mrg abd distended soft bs not obviously tender ext no le edema just removed stockings skin no rashes lines ngt pegj tube pertinent results past cultures sputum gram stain gram coccioropharyngeal flora cmv vl negative hsv vzv direct antigen negative toxo igg positive igm negative croccal antigen negative bal pcp respiratory viruses adeno parainfluenza influenza rsv legionella afb all negative cultures from this admission urine ngtd blood ngtd blood ngtd csfspinal fluid ngtd gram stain final no polphonuclear leukocytes seen no microorganisms seen csfspinal fluid cryptococcal antigen not detected urine ngtd sputum pmns and epithelial cellsx field per x field gram positive cocci in pairs and clusters per x field gram positive cocci in pairs and chains grew out urine ngtd blood ngtd blood ngtd urine ngtd laboratory data pm blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood fdp am blood fibrino am blood ret aut am blood glucose urean creat na k cl hco angap pm blood glucose urean creat na k cl hco angap am blood ldldh totbili dirbili indbili pm blood alkphos amylase totbili am blood lipase pm blood lipase am blood ckmb ctropnt am blood ckmb ctropnt pm blood ckmb ctropnt am blood ckmb mtwice day ctropnt am blood calcium phos pm blood albumin calcium phos am blood osmolal am blood cortsol am blood cortsol am blood phen phenyfrpnd pm cerebrospinal fluid csf wbc rbc polys lymphs monos pm cerebrospinal fluid csf totprot glucose brief hospital course yo with histor astroca dvt recent admission for pcp pna requiring intubation transferred from withdx aspiration pna after he was found unresponsive febrile pt was intubated at for his asp pna extubated h later transferred to for further care his course at was complicated by ugib from pegj tube erosion into duodenum intubation for egd and fast extubation probable nonconvulsive seizures ms change initially the patients poor mentation was felt to be resolving infection asp pna in the setting low threshold for confusion because his underlying astroca he was continued on antibiotics see below imaging showed that his astroca had not advanced and was not in and itself the cause for his decreased mental capacity the possibility was also raised that there was contribution to his confused state from the steroids that the patient received on his arrival to unclear reason why possibly initial concern that ms change was tumor progression no explanation in dc summary the patient had been on high dose steroids prior to his previous admission but these had been tapered over approx three wks and his mental status had improved several stim tests during this admission showed the patients adoctornal function to be intact he was continued on low dose dexamethasone qday while in house patient was evaluated for nonconvulsive seizure disorder by eeg with prior study showing subcortical dysfunction but not clear evid sz activity first stud this admission showing cortical and subcortical dysfunction no clear evid sz second eeg from this admit on showed paroxysmal semirhythmic epileptiform activiter both frontal regions the patient was started on dilantin and keppra at this time final eeg during this admit on showed an area cortical hypersynchrony likely to be focus sz activity though no such sz activity was observed during the study the patients ms seemed to improve slightly as his dilantin and keppra doses were increased though eventually he plateaued in state minimal responsiveness aspiration pneumoniapneumonitis the patient had at least two likely aspiration events one prior to his admission to when he was found at with tube feeds all over his clothing and second witnessed event during bronchoscopy at pt had completed d course clindaprimaquine and pcp smear was negative at from sputum was felt to be likely colonization given recent hospitalizations and rehab stay patients respiratory failure was felt to be most likely to pneumonitis or bacterial pneumonia after admission he was continued on cefipime started at and flagyl was added given witnessed aspiration at to cover asp flora vancomycin was started to cover from sputum id was consulted rec lp atovaquone ppx for pcp developed that was presumed from cefepime so this was stopped initial series cxrs showed stable diffuse patchacification the right lung patients antibiotics were changed multiple times during his admission on transfer from the patient was on cefepime and levaquin immediately the levaquin was dcd and flagyl was added then on pt was noted to have reticular trunk and thighs thought to be doctorg cefepime was dcd and pt was started back on levofloxacin again instead patient was on the following abx while in house vanc d levaquin d clinda d flagyl d patient was off all antibiotics for three days then noted to have lll asp pna vs pneumonitis on and so levaquin and flagyl were started again on dvt neg cta and lenis here during previous admission fragmin started at was dcd on admission and pt maintained on prophylactic dose heas needed then patients lle was noted to be swollen on so lenis were done that showed bl dvts in superficial and common femoral veins patient was started on heas neededgtt anticoagulation he was then transitioned to lovenox but both were stopped in the setting hct doctorp shortly thereafter with bleeding from his pegj site as patient was poor candidate for anticoagulation ir placed an ivc filter patient was later noted to have lue dvt but was not anticoagulated for this given his continued slow hct doctorp and likely continued ugib bleed if his hct stabilizes in the future and he can be reegd anticoagulation might be indicated however this is unlikely to happen in the near term given his continued gi bleeding and hct doctorp astroca repeat ct scan showed large mass in temporal lobe extending into frontal lobe no significant change since prior study with regard to the mass effect on lateral ventricle no evid brainstem compression patients astroca was not felt to have changed on this admission and as result his symptoms were not felt to be directly related to this lesion it was felt that his tumor may have lowered his threshold for becoming confused somnolent but that either infection or seizures were the cause his altered mental status patient was continued on low dose dexamethasone as well as dilantin and keppra gi bleed the patient had peg placed on in the setting anticoagulation he then began to have melena external bleeding from his pegj site and received transfusions prbc without adequate hct increase he was transferred to the for intubation for the purposes preforming an egd egd showed bleeding around the pegj and string associated with clot and fresh blood in the duodenum the pegj tube was removed by ir over wire and peg tube was placed following the procedure the patient had an ivc filter placed as he has had dvt and given his bleeding anticoagulation was not an option he continued to have doctorpping hct after the procedure and was transfused several more units prbcs over the remainder his admission he was noted to still have gastrografinpositive residuals on in the setting spoke with gi at that point who said that they would not reegd and reintubate the patient unless his bleeding became brisk again they rec to continue ppi supported with transfusions as necessary patient required total units before egd units while in for egd then units after back on the floor for slow hct doctorp with guaiac positivity medications on admission meds on admission senna docusate acetaminophen sertraline qday albuterol neb qh prn levetiracetam dexamethasone iv qh esomeprazole qday atovaquone qday dalteas neededu sc bid metoprolol tid ipratroprium neb once dayvancomycin g qh plan to complete d course cefepime g qh plan to complete d course discharge medications ipratropium bromide solution sig one nebulizer treatment inhalation qh every hours nystatin unitml suspension sig five ml po once day times day atovaquone ml suspension sig five ml po bid times day ml acetaminophen tablet sig tablets po qh every to hours as needed metoprolol tartrate tablet sig one tablet po bid times day dexamethasone tablet sig two tablet po daily daily docusate sodium ml liquid sig ten ml po bid times day senna tablet sig one tablet po bid times day lactulose g ml syrup sig thirty ml po bid times day levetiracetam tablet sig three tablet po bid times day phenn ml suspension sig eight ml po qam qpm ml phenn ml suspension sig twelve ml po qnoon albuterol sulfate solution sig one treatment treatment inhalation qh every hours heas neededflush picc unitsml ml iv dailyprn ml ns followed by ml unitsml heas needed units heas needed each lumen daily and as needed inspect site every shift sodium chloride flush ml iv dailyprn peripheral iv inspect site every shift sodium chloride flush ml iv dailyprn picc please flush picc line with ns inspect site every shift heas neededflush picc unitsml ml iv dailyprn ml ns followed by ml unitsml heas needed units heas needed each lumen daily and as needed inspect site every shift pantoprazole sodium recon soln sig one recon soln intravenous qh every hours discharge disposition home with facility healthcare discharge diagnosis altered mental status aspiration pneumonia nonconvulsive seizures upper gi bleed from pegj tube erosion hypernatremia constipation discharge condition stable though with continued decreased level responsiveness discharge instructions please continue to take all medications as prescribed and to follow up with doctor and the rest your healthcare team if you develop worsening shortness breath high residuals your tube feeds bleeding from anywhere black stool or anher concerning findings please call doctor office immediately for instructions on how to be reevaluated followup instructions please touch base with doctor in the week after your discharge to schedule follow up appointments'

In [6]:
# sentence = 'patient recorded as having no known to doctorgs attending chief complaint renal mass major surgical or invasive procedure right laparoscopic radical nephrectomy yf with cad and esrd on hd with nearly cm left renal mass concerning for malignancy diabetes type cb retinopathy neuropathy pt not currently taking meds due to insurance issue reactive airway disease asthma py smoking and likely copd depression histor pulmonary nodules consistent with calcified granuloma menorrhagia hypertension poorly controlled histor hypertensive urgency hypercholesterolemia chronic lower back pain cri most recent cr values in the s thd mass never followed up with biopsy osteoporosis left renal mass concerning for rcc but never followed up for biopsy chronic anemia thought ckd not on epo mgus chf tte demonstrated moderate to severe global left ventricular hinesis lvef moderate mr and moderate pulmonary artery hypertension lives with roomate in in apt she grew up with her family as carnival worker and traveled with them illiterate smokes ppd has pack yr history no etohillicits on ssi currently only family support seems to be her in ny multiple family members with dm mi cva uncle and two cousins had kidney disease requiring dialysis mother with breast cancer physical exam avss gen nad abd soft nt nd incisions cleandoctorintact brief hospital course patient was admitted to after undergoing laparoscopic right radical nephrectomy no concerning intraoperative events occurred please see operative note for details the patient received perioperative antibiotic prophylaxis the patient was transferred to the icu in stable condition for monitoring due to extensive comorbidities and not for any intraoperative complication on pod pain was well controlled on pca htorted for urine output cchour provided with pneumoboots and incentive spirometry for prophylaxis and ambulated once she was seen by the cardiology and the nephrology and underwent hd while int he icu on pod the patient was restarted on home medications basic metabolic panel and complete blood count were checked pain control was transitioned from pca to oral analgesics diet was advanced to clearstoast and crackers diet on pod foley was removed without difficulty and diet was advanced as tolerated prior to discharge the patient underwent ct scan with contrast the chest to evaluate pulmonary nodules seen on cxr she underwent hd after this the remainder the hospital course was relatively unremarkable the patient was discharged in stable condition eating well ambulating independently voiding without difficulty and with pain control on oral analgesics on exam incision was clean doctor and intact with no evidence hematoma collection or infection the patient was given explicit instructions to followup in clinic with doctor in weeks and to also call for followup with nephrology and cardiology discharge medications docusate sodium capsule sig one capsule po bid times day disp capsules refills acetaminophen tablet sig two tablet po qh every hours asas needed tablet chewable sig one tablet chewable po daily daily omeprazole capsule delayed releasee sig one capsule delayed releasee po daily daily disp capsule delayed releasee refills lisinopril tablet sig two tablet po daily daily disp tablets refills simvastatin tablet sig two tablet po daily daily disp tablets refills complexvitamin cfolic acid capsule sig one cap po daily daily calcium acetate capsule sig two capsule po tid withmeals times day with meals htormorphone tablet sig tablets po every four hours as needed for pain disp tablets refills gabapentin capsule sig two capsule po once dayeach hemodialysis disp capsules refills metoprolol succinate tablet sustained release hr sig one tablet sustained release hr po once day disp tablet sustained release hrs refills lasix tablet sig one tablet po once day disp tablets refills discharge disposition home discharge diagnosis renal mass discharge condition stable discharge instructions you may shower but do not bathe swim or immerse your incision do not lift anything heavier than phone book pounds or doctorve until you are seen by your urologist in followup tylenol should be your first line pain medication narcotic pain medication has been prescribed for breakthough pain replace tylenol with narcotic pain medication max daily tylenol dose is gm note that narcotic pain medication also contains tylenol acetaminophen do not doctorve or doctornk alcohol while taking narcotics colace has been prescribed to avoid post surgical constipation and constipation related to narcotic pain medication discontinue if loose stool or diarrhea develops resume all your home medications except hold nsaid asas needed and ibuprofen containing products such as advil & motrin until you see your urologist in followup call your urologists office today to scheduleconfirm your followup appointment in weeks and if you have any questions if you have fevers vomiting or increased redness swelling or discharge from your incision call your doctor or go to the nearest er do not eat constipating foods for weeks doctornk plent fluids followup instructions please call doctor office for followup please call your cardiologist for followup completed by'

In [ ]:
# 2) bigger model: positive, missclass as negative, missclass as negative, missclass positive 

In [6]:
# sentence = 'medicine iodine ct contrast attending chief complaint weakness major surgical or invasive procedure intubation mechanical ventilation extubation incision and doctorinage right axillary abscess placement central venous catheter the patient is yo with histor metastatic melanoma with spine brain and adoctornal mets who has had left sided weakness for months and has been developing right sided weakness over the past week thought to be related to known c mass presented to the ed today due to inability to perform adls the patient was last seen by his oncologist doctor on at that time he was started on tpch chemotherapy regimen day additionally the patient was recently seen by his pcp for the left sided weakness and his dexamethasone regimen was increased to qh with minimal improvement he denies any loss bowelbladder continence no other neurological sx denies feverchills has had significant diffulty maintaining appropriate glucose levels since starting on steroids note the patient has also developed right axillary abscess over the past week started on keflex on by np at oncologists office this has been increasing in size began spontaneuosly doctorining today when palpated by paramedics en route to the ed the patients fs was noted to be and he received humalog at that time in the ed the patients intial vitals were hr bp rr and spo ra laboratory studies revealed na and given amp calcium gluc and an ecg was performed showing without peaked ts cxr showed no acute intrathoracic process received home insulin and became slightly hlycemic had bp systolic and received l ivf with good response got vanc x for his abcess iv in ed infiltrated and its unclear how much the above the patient received on the floor the patient is mentating well somewhat hensive with bas neededing from systolic otherwise no acute complaints ros as per hpi otherwise denies any cp palp sob nvd feverchills changes in bowelbladder habits weight loss ha or vision changes oncologic history he biops mm thick primary melanoma without ulceration from his right mid back in he sentinel lymph node biopsy without evidence melanoma in his right axillary nodes he was enrolled in ecog protocol randomized to the observation arm he was well until when cxr revealed multiple new pulmonary nodules with ct confirmation he was referred to interventional pulmonary doctor for bronchoscopic biopsy brain mri on revealed multiple brain mets and petct on revealed widespread metastatic disease including worrisome cervical spine lesion on he cspine mri confirming intramedullary metastasis at cc he was seen urgently by doctor orthopedics and felt not to be surgical candidate he bronchoscopy and left adoctornal biops with the left subcarinal ln showing no malignant cells and the left adoctornal bx cw metastatic melanoma he completed day course cspine radiation at hospital on he began ipilimumab on the brain metastasis trial on and did very well he had slow growth in the lll lung lesion and or bronchoscop with for bronchoscopy biopsy and debridement tumor pathology confirmed metastatic melanoma he vats lll wedge resection by doctor on pathology confirmed melanoma in the lung lesion but margins were clear and lymph nodes were negative for tumor cspine mri was performed in to evaluate lue numbness noted intermittently since rigid bronchoscop with slight increase in size cc mass felt rt inflammation from ipilimumab he subsequently developed progressive left shoulder arm and neck pain and increased ue numbness prompting er evaluation on cspine mri revealed slight increase in the cc mass and increased edema and he was started on prednisone his brain mri and torso ct were stable without disease progression tumor is braf wild type he was removed from the ipilimumab brain metastasis clinical trial due to symptoms and interval increase in the cc mass although ipilimumab related inflammation rather than true disease progression was felt to be possible attempts to lower steroid dosing were unsuccessful started on temodar diabetes diagnosed in hypertension hypercholesterolemia vasectomy he is retired from the insurance industry and also taught computer at private middle school on pack year smoker quit in doctornks per month no doctorgs had an uncle who died metastatic melanoma his grandmother had breast cancer and his brother died an mi in his sleep at age his sister also has cardiac problems physical exam admission physcial exam vitals l general appears well and in nad joking in bed heent perrla eomi anicteric mm doctor op clear cv rrr s and s no mrg lung ctab no withrr abdomen soft ntnd bsx extremity two areas erythema on the right anterior chest superior lesion is fluctuant and doctorining size is approx cm neuro awake alert and oriented strength in lue and lle in rue and rle some numbmness to palpation diffuseler extremities discharge physical exam vs bp hr rr spo on ra general nad aox heent doctor mm op clear no oral ulcerations or exudate neck no jvd cv rrr no mrg lungs clear to auscultation anteriorly no crackles or wheezes abd bs ntnd soft ext wwp soft pitting edema bilaterally no clubbing or cyanosis right axillary wound with bandage cdi neuro strength in the uele on the right strength in the uele on the left pertinent results admission labs pm blood wbc rbc hgb hct mcv mch mchc rdw plt ct pm blood neuts bands lymphs monos eos baso atyps metas myelos pm blood pt ptt inrpt am blood fibrino pm blood glucose urean creat na k cl hco angap pm blood alkphos totbili pm blood alkphos totbili am blood ldldh ckcpk pm blood lipase pm blood albumin am blood calcium phos pm blood osmolal am blood typemix po pco ph caltco base xs pm blood glucose na k cl calhco am blood lactate discharge labs am blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood neuts bands lymphs monos eos baso atyps metas myelos am blood pt ptt inrpt am blood glucose urean creat na k cl hco angap am blood ldldh alkphos totbili am blood albumin calcium phos other relevant labs am blood vanco am blood am blood t free t pm urine hoursrandom creat na k cl pm urine osmolal microbiology pm blood culture source lineleft sc blood culture routine final pseudomonas aeruginosa final sensitivities pseudomonas aeruginosa cefepime ceftazidime ciprofloxacin gentamicin meropenem piperacillintazo tobramycin am swab site right axilla gram stain final per x field polphonuclear leukocytes per x field gram negative rods anaerobic culture final no anaerobes isolated fungal culture preliminary no fungus isolated wound culture final pseudomonas aeruginosa heavy growth enterococcus sp rare growth pseudomonas aeruginosa enterococcus sp ampicillin cefepime ceftazidime ciprofloxacin gentamicin meropenem penicillin piperacillintazo tobramycin vancomycin imaging studies chest pa & lat at pm findings the lungs are clear without consolidation or edema the mediastinum is unremarkable the cardiac silhouette is within normal limits for size no effusion or pneumothorax is noted degenerative changes are seen throughout the thoracic spine impression no acute pulmonary process mr at pm findings the intramedullary heterogeneously enhancing lesion at cc level is stable in size and morphology currently measuring craniocaudal anteroposterior coronal mm likewise the associated cord edema is unchanged notably extending from c to c level at the level the lesion the spinal cord is expanded and there is unchanged severe spinal canal stenosis with loss csf signal stenosis at this level is exacerbated by unchanged central disc protrusion there is no interval change with regard to multilevel multifactorial degenerative changes most notably with moderate neural foraminal stenosis on the left at cc level and multilevel disc bulges at cc cc and cc no evidence new metastatic lesions the visualized paraspinal soft tissues are unremarkable impression stable appearance cervical intramedullary metastatic lesion with extensive cord edema cta abdomen pelvis at pm ct abdomen there is moderate subcutaneous emphysema along the right anterior chest wall presumably postprocedural the lung bases are clear there is no pleural effusion the heart is normal in size without pericardial effusion there is multivessel coronary arterial disease intraabdominal visceral evaluation is highly limited on current examination due to significant streak artifacts however allowing for such there is no focal liver lesion the gallbladder spleen and adoctornal glands are unremarkable bilateral kidneys enhance symmetrically without htornephrosis or htorureter an enteric tube is in place with tip coiled within the stomach small and large bowel loops are normal in caliber without wall thickening or obstruction there is no evidence active intestinal hemorrhage on current examination extensive colonic diverticulosis is present without diverticulitis hepatic arterial anatomy is conventional the celiac trunk sma and renal arteries are patent is also patent moderate atherosclerotic calcifications are present in the infrarenal aorta ct pelvis the bladder is partially collapsed containing nondependent air likely related to foley catheter placement the rectum appears within normal limits small bilateral fatcontaining inguinal hernias are present there is no pelvic sidewall or inguinal lymphadenopathy by size criteria subcentimeter external iliac and inguinal lymph nodes are stable cm mass in the right upper posterior chest wall soft tissues a is similar as compared to bone window no focal concerning lesion impression no ct evidence active lower gi bleed colonic diverticulosis likely postprocedural right anterior chest wall subcutaneous emphysema correlation with any recent histor instrumentation is recommended stable right upper posterior chest wall soft tissue mass unchanged subcentimeter iliac and inguinal lymph nodes mr at pm findings cervical vertebrae are normal in height and alignment craniocervical junction appears normal as compared to the previous mri from there has been no interval change in the size and morpholog intramedullary enhancing lesion at cc level again noted is extensive cord edema surrounding the lesion extending from c to c levels multilevel degenerative changes in the cervical spine are unchanged no new metastatic lesions are seen pre and paravertebral soft tissues are unremarkable impression no interval change in cervical intramedullary metastatic lesion with unchanged spinal cord edema bilat lower ext veins at pm findings the common femoral superficial femoral and the popliteal as well as the deep veins the calves on both sides show normal ultrasound appearance compressibility and doppler waveforms conclusion the ultrasound examination was negative for dvt in either lower extremity brief hospital course hensionhyperglycemia first icu course upon admission the initial concern was for sepsis in the setting an axillary abscess patients blood pressures responded well to fluids and he stabilized quickly in the ed patients blood glucose was so likely he was simply volume depleted secondary to osmotic diuresis in his hyperglycemic state he also had numerous electrolyte abnormalities including hatremia and hyperkalemia which were corrected prior to transfer to the floor second icu course documented under gi bleed see below three times dayicu course on hospital day patient developed hension and tachycardia in the setting missing doses unasyn due to loss iv access he was intubated for airway protection transfered back to the icu and fluid resuscitated his antibiotics were broadened to vanczosyn and he responded well hemodynamically he was able to be extubated after less than hours on his three times dayicu course his hension was attributed to sepsis in the setting pseudomonas bacteremia which was thought to be due to infection the right axillary abscess with pseudomonas also isolated from culture the axillary wound axillary abscess the patient developed an axillary abscess week prior to admission which started spontaneously doctorining the da admission the patient had been on keflex since and received dose vancomycin in ed the patients presentation was likely related to patients immunosupression with chronic steroids and chemo the patient was covered with vancomycin and zosyn initially surgery incised and doctorined the abscess infectious disease recommended vancomycin and unasyn after the patients three times dayadmission to the the axillary abscess was noted to have grown pseudomonas and enterococci he was initially restarted on vancomycin and zosyn as susceptibilities returned the patient was treated with vancomycin and ciprofloxacin gi bleeding on hospital day patient was noted to pass brbpr and clots per rectum in setting platelets he was transiently hensive to the s systolic but recovered his pressures to the low s high s he was transferred to the where ng lavage returned clear gastric contents he was transfused units prbc the source the bleeding was presumed to be from diverticular bleed given his neutropenia the patient was not scoped patient had cta the abdomen and pelvis showed no evidence active lower gi bleed but the presence colonic diverticulosis he stabilized and improved with platelet transfusion ddavp ivf and prbc while in the unit on the floor the patient had no further episodes gi bleeding rue weakness most likely due to the patients c vertebral mass and related nerve impingment no other signs suggestive cord compression initially changed dexamethasone to htorcortisone due to hension and electrolyte abnormalities but switched back to patients original steroid dosing once he was stabilized obtained mri cspine and brain which did not show interval progression metastatic disease though the study was technically limited the patient right upper and lower extremity weakness improved through his hospital course while working with pt and ot acute kidney injury prior to hospitalization patient had baseline creatinine on presentation his creatinine was and continued to climb to elevated creatinine was attributed to prerenal status the patients serum creatinine improved with fluids to at the time transfer to the oncology floor his serum creatinine remained stable through the remainder his hospitalization melanoma when the patient initially presented he had been started on new chemotherapy regimen per hematologyoncology recs tpch chemotherapy was held because acute kidney injury the patients chemotherapy was held through the admission the patient did undergo an mri the brain and cervical spine to monitor the patients disease and it did not show interval progression disease diabetes mellitus type patients finger stick blood glucose were checked with meals and prior to bedtime initially the patient refused to be managed by nurses for insulin and wanted to take his home medications he later agreed to follow an insulin sliding scale and was started on insulin glargine as well his blood sugars were difficult to control during his stay due to his continued high dose steroids his evening lantus dose was uptitrated several times but he continued to have elevated fingersticks at the time discharge his fbgs were running in the s he will need close followup from endocrinology after discharge for further adjustment his insulin regimen hypertension given the patient initial presentation hension his home lisionpril was held during his hospitalization and discontinued at discharge hyperlipidemia his home rosouvastatin was continued through the admission thrombocenia patient initially presented with thrombocenia thought to be due to chemotherapy he received platelets early during his admission to prevent gi bleeding with transfusion goal to keep platelets greater than the patients platelet count recovered later in his hospitalization hhdism patients home dose synthroid was continued through much the hospitalization was doctorwn that returned elevated but with normal free t no changes were made to the patients synthroid he will need repeat tfts weeks from discharge as well as endocrine followup elevated ldh his ldh rose since his last admission to the his hematocrit remained stable as has his tbili suggesting that the elevated ldh was not due to hemolysis the patients and were within normal limits previously were trending down in the setting shocked liver secondary to sepsis which does not suggest liver pathology given the patients dexamethasone it was possible that the rise in ldh may represent pcp infection though the patient denies shortness breath and he had been saturating well on room air ldh has also been used as tumor marker for melanoma so it is possible that the rise may present the patients underlying metastatic disease because the patients prolonged dexamethasone course he was started on pcp prophylaxis with bactrim ds hatremia patient was initially admitted to the with hatremia thought to be due to poor oral intake urine lytes were consistent with siadh in the setting his known metastatic melanoma upon the patients second course his sodium improved with stress dose steroids on the floor as his dexamethasone was being weaned his serum sodium was noted to be falling his dexamethasone was increased back to po qh he was briefly treated with fludoctorcortisone but this was discontinued at discharge endocrine consult felt that an adoctornal or thd etiology for his hatremia was unlikely but that he will need slow taper his steroids given his prolonged high dose course his na was fairly stable around at discharge he will need to follow up with endocrinology soon after discharge for management his steroid taper transition care continue vancomycin and ciprofloxacin for total weeks after negative blood culture on consider infectious disease followup for further antibiotic recommendations monitor serum sodium initially every days and then weekly as sodium levels stabilize endocrinology followup within weeks discharge for hhdism diabetes management and steroid taper slow steroid taper over several months per endocrine recs repeat tfts weeks from discharge continue to work with pt and ot at rehab oncology followup as scheduled soon after discharge medications on admission dexamethasone tablet tablets by mouth every six hours take wtih tablet dexamethasone tablet tablets by mouth hours take with tablet hdoctorxyurea take tabs every hrs for times and then tabs every hrs for nine times insulin lispro protam & lispro humalog mix prescribed bher provider unitml suspension up to units with meals insulin lispro humalog prescribed bher provider unitml cartridge cover dose levothine levothroid mcg tablet tablets by mouth once day taken in the morning lisinopril tablet tablets by mouth once day lorazepam tablet tablets by mouth at bedtime as needed for insomnia lormustine tabs capsules once for total dose nystatin unitgram powder apply topically twice day as needed for groin rash omeprazole prescribed bher provider capsule delayed releasee capsules by mouth daily daildansetron hcl tablet tablets by mouth take tab for nausea as needed no substitution oxone tablet tablets by mouth times daily as needed procarbazine matulane capsule capsules by mouth take tabs every hrs take tabs every hrs for times and tabs every hrs for times no substitution rosuvastatin crestor prescribed bher provider tablet tablets by mouth once day testosterone testim gram gel topically daily thioguanine tabloid tablet tablets by mouth tabs every hrs take tabs every hrs for times then tabs every hrs for times no substitution asas needed tablet discharge medications levothine mcg tablet sig one tablet po daily daily ciprofloxacin in dw ml piggyback sig one dose intravenous qh every hours vancomycin recon soln sig recon solns intravenous h every hours docusate sodium capsule sig one capsule po bid times day sulfamethoxazoletrimethoprim tablet sig one tablet po monday wednesday friday dexamethasone tablet sig one tablet po qh every hours oxone tablet sig tablets po qh every hours as needed for pain rosuvastatin tablet sig one tablet po once day acetaminophen tablet sig tablets po qh every hours as needed for fever or pain simethicone tablet chewable sig one tablet chewable po once day times day as needed for gas collagenase clostridium hist unitg ointment sig one appl topical daily daily applied to gluteal wound senna tablet sig one tablet po bid times day as needed for constipation omeprazole capsule delayed releasee sig one capsule delayed releasee po once day lantus unitml solution sig forty six units subcutaneous at bedtime humalog unitml solution sig one injection subcutaneous four times day before meals and at bedtime according to sliding scale discharge disposition extended care facility hospital and islands discharge diagnosis primary diagnosis pseudomonas bacteremia and sepsis right axillary abscess lower gastrointestinal bleeding steroid induced hyperglycemia hatremia secondary diagnosis metastatic melanoma diabetes mellitus type hhdism discharge condition mental status clear and coherent level consciousness alert and interactive activity status out bed with assistance to chair or wheelchair discharge instructions dear mr it was pleasure taking care your during your hospitalization at during this hospitalization you were admitted to the icu for concerns sepsis you were stabilized and then transferred to the oncology floor you then developed bleeding from your gi tract with low blood pressure and had to be transferred back to the icu for stabilization you had three times darse in the icu because pseudomonas infectious the right axilla that spread into your blood you are currently being treated with antibiotics for your infection please take all medications as instructed the following medication changes have been made start ciprofloxacin iv every hours start vancomycin iv every hours start bactrim ds tab on monday wednesday friday start lantus insulin units at bedtime start humalog insulin according to sliding scale changed levothine mcg by mouth daily stop lisinopril by mouth daily you will need close followup with endocrinology for your diabetes hhdism and steroid taas needed you will also need infectious disease followup for management the antibiotics course for your axillary abscess and recent bacteremia you also have followup scheduled with your oncologist next week please keep all appointments as listed below followup instructions department hematologyoncology when tuesday at pm with np twice dayng campus east best parking garage department hematologyoncology when tuesday at pm with doctor twice dayng sc clinical ctr campus east best parking garage department radiology when tuesday at am with radiology mri twice dayng campus east best parking garage md'

In [7]:
# sentence = 'medicine percocet attending chief complaint sobcough major surgical or invasive procedure central line placement arterial line placement liver biopsy the patient is yo with histor aml status post matched related allogeneic stem cell transplant on with busulfan and can conditioning with complications cmv viremia gvhd hemorrhagic cystitis and disseminated tb with recent admissions with pericardial effusion and tamponade status post window on the patient presented to the ed today with sob and cough week the patients wife reports that over the past days the patient has had fevers to fatigue weakness and persistent cough with blood tinged sputum note the patient was recently admitted on with shortness breath cough and peripheral edema he was found to have pericardial effusion with early tamponade physiology the effusion was doctorined for total ccs fluid it was felt the more likely etiology as viral process although no specific etiology has been found recent afb smears have been negative but recent culture has grown positive for xanthamonas he also was noted for persistent positive bglucan and mr was treated with bactrim to cover both pcp and unfortunately the patient required another urgent admission with pericardial effusion and tamponade this was doctorined again and window was place there was no infectious etiology identified and the thought was this could be related to gvhd during this admission his antibiotics were changed to prophylactic doing for pcp with bactrim since that time the patient has been slowly improving and was about to restart photopheresis therapy for gvhd in the ed today initial vitals were t hr bp rr o nrb ra initially found to have glucose he received amp d he became hensive to the s and neosyneas needed was initialed he was them emergently intubated received etomidate succ his hr remained in the ss and he was cardioverted j and converted to sinus tacycardia hr ss rij was placed for code sepsis his bp remained low and levophed was started he received l ns vanocmycin ceftaz flagyl and levofloxacin cxr was concerning for rul pna and fluid overload cardiology was consulted and performed bedside echo which revealed loculated effusion unchanged from prior echo on he also received unit ffp after the line placement onc history per omr diagnosed in early with nightly fevers bm bx revealed aml flow cetry showed aberrant expression cd cd hladoctor cd dim cd cd and cd ct scan revealednecrotic lymph nodes in the superior mediastinum and periportalregion and multiple low attenuation lesions in the liver and spleen concerning for microabscesses from disseminated infection induction chemotherapy with cytarabine and idarubicin complicated by persistent fevers and extensive workup ultimately revealing disseminated tuberculosis infection his course was also complicated by rapid atrial fibrillation and hension and the development severe cardiomyopathy status post one dose highdose arac at per meter squared lowered dose due to his disseminated tuberculosis and then he received second course hidac at gram per meter squared dose and developed acute onset gait instability no further chemotherapy given relapsed in reinduction with me on noted for pulmonary nodules which were suspicious for aspergillus and empirically treated with voriconazole with improvement noted on ct admitted on for maintenance therapy while awaiting bmt however upon admit he was again found to have blasts he proceeded with idarubicin and cytarabine butdid not achieve remission status post high dose arac with remission sibling related allo transplant on allo course cb increased lfts unclear etiology possibly from chemotherapy renal failure attributed to csa and received only dose mtx due to mucositis post transplant course complicated by asymptomatic cmv viremia and viraluri syndoctormes in developed diarrhea with eo gvh on endoscopy he also had hematuria but no evidence bk virus he started photopheresis diarrhea abated but lfts rose therapy attempted for gvh liver using pulse prednisone and increase in cellcept with stabilization but no significant improvement received pentostatin on liver biopsy cw gvhd started rituxan for weeks in nononc pmh disseminated tb status post treatment with inh levofloxacin and rifabutin hypertension and heart murmur diabetes mellitus type chemo related heart failure and cardiomyopathy ef histor atrial fibrillation recent ekgs in nsr cmv viremia he is married and lives at home with his wife & childoctorn he is machine operator but is currently not working he immigrated from in early he smoked approximately cigarettes per day for years and stopped year ago he does not doctornk alcohol notable for mother who passed awa myocardial infarction his father passed awa liver disease he has four living brothers and two living sisters all in good health physical exam vitals ac x fio general intubated sedated unresponive heent pupils mm and reactive neck elevated jvp cv tachycardic no murmur no rub lungs crackles at bases abdomen soft ntnd ext pitting edema bl skin pale changes consistent with gvhd pertinent results am gran ct am pt ptt inrpt am plt smrrare plt count am hhrom anisocyt poikilocy macrocyt microcytnormal polychrom target schistocy stippled teardoctorp pappenhei am neuts bands lymphs monos eos basos atyps metas myelos nuc rbcs am wbc rbc hgb hct mcv mch mchc rdw am calcium phosphate magnesium am ckmbnotdone am ctropnt am ckcpk am glucose urea n creat sodium potassium chloride total co anion gap am freeca am glucose lactate na k cl am ph commentsgreen top am urine granular hyaline am urine rbc wbc bacteriamod yeastnone epi trans epi am urine bloodlg nitriteneg protein glucosetr ketonetr bilirubinmod urobilngnneg ph leukneg am urine coloramber appearhazy sp am typeart rates tidal vol po pco ph total co base xs intubatedintubated am hgb calchct o sat am commentsgreen top am freeca am lactate am typeart rates tidal vol o po pco ph total co base xs aado req o assistcon intubatedintubated am pt ptt inrpt am plt count am wbc rbc hgb hct mcv mch mchc rdw am calcium phosphate magnesium am ckmbnotdone ctropnt am altsgpt astsgot ldldh ckcpk alk phos tot bili dir bili indir bil am glucose urea n creat sodium potassium chloride total co anion gap am plt count pm urine gr holdhold pm urine osmolal pm urine hoursrandom pm urine hoursrandom urea n creat sodium pm typeart temp rates tidal vol peep o po pco ph total co base xs assistcon intubatedintubated pm typeart temp rates tidal vol peep o po pco ph total co base xs assistcon intubatedintubated ruq ultrasound perihepatic ascites and gallbladder wall edema which are without the presence cholelithiasis or biliary dilatation the question three times dayspacing or even hepatitis should be considered known peripancreatic lymphadenopathy echo due to suboptimal technical quality focal wall motion abnormality cannot be fully excluded overall left ventricular systolic function is severely depressed estimation ejection fraction is impossible due to limited images although lv function appears significantly more depressed than previously there is abnormal septal motionposition there is small pericardial effusion the effusion appears loculated there are no echocardiographic signs tamponade pericardial constriction cannot be excluded echo impression normal biventricular cavity size with mild global left ventricular hinesis cw diffuse process toxin metabolic etc mild aortic regurgitation with normal valve morphology anterior organized pericardial effusion without evidence for hemodynamic compromise compared with the prior study images reviewed left ventricular systolic function is slightly improved previouslerestimated the anterior pericardial effusion is similar ct abdpelvis multifocal airspace disease as demonstrated on multiple recent chest radiographs no evidence bowel obstruction moderate ascites increased compared to prior ct study from overall stable retroperitoneal and periportal lymphadenopathy right colonic wall thickening including the caecum consistent with non specific colitis no associated abscess or freecontained air several liver calcifications suggesting prior granulomatous disease brief hospital course mr is yo with histor aml status post matched related allo transplant on with busulfan and can conditioning with complications cmv viremia gvhd hemorrhagic cystitis and disseminated tb with recent admissions with pericardial effusion and tamponade status post window on admitted with respiratory failure and hension secondary to pneumococcal sepsis respiratory failure intubated in ed vent setting were weaned to pressure support with minimal support for several weeks could not extubate secondary to poor mental status in his last few days switched back to assist control for more ventilatory support extubated approximately minutes prior to death pneumococcal sepsis completed week course ceftriaxone followed by meropenem acute renal failure massive renal insult from hension from pneumococcal sepsis had days cvvh in an attempt to improve his mental status fib with rvr histor fib with rvr cardioverted in ed was shocked more times in icu eventually loaded with amiodarone which was stopped after lfts started to climb rate controlled after amio load aml continued on atovaquone acyclovir until copies cmv noted in blood and posaconazole transfused as needed gvhd has gvhd liver skins eyes held cellcept initially restarted during admission weaned from stress dose steroids initially then given high dose steroids when liver function began to deteriorate liver failure thought secondary to shock liver gvhd liver biopsy performed on demonstrating minimal portal three times day but generally burned out liver from gvhd no infection identified in the last several days his pressor requirement increased and he required more support from the ventilator after discussion with his primarcologist and the family he was made cmo and expired shortly therafter at pm on medications on admission docusate sodium nystatin five ml po once daybenzonatate tid prednisone daily posaconazole ml tid pyridoxine daily vitamin unit daily folic acid daily lantus units qam ss toprol xl daily trimethoprimsulfamethoxazole one tablet po mwf mhenolate mofetil ammg pm dexamethasone ml swish po bid acyclovir tid atovaquone ml suspension ml suspensions by mouth once day lumigan doctorps doctorp in the right eye twice day restasis doctorpperette one doctorp in each eye twice day methylcellulose doctorps one doctorp in each eye four times day omeprazole dailtin oxone tab q prn prednisolone doctorps suspension one doctorp in each eye once day discharge medications expired discharge disposition expired discharge diagnosis expired discharge condition expired discharge instructions expired followup instructions expired'

In [8]:
# sentence = 'neurology bactrim furosemide cefepime attending chief complaint altered mental status major surgical or invasive procedure egd with injection epineas needed into duodenum for hemostasis hpi yearold gentleman recently diagnosed by biopsy with gliomatosis cerebri grade ii astroca in status post xrt temodar patient was recently admitted at for pcp pneumonia which he developed in the setting increasing symptoms course cb hic respiratory failure requiring intubation transfer to icu during icu course developed hension requiring pressors occurring in the setting hypersensivity reaction to bactrim and one episode hematochezia during anticoagulation for previous pe following transfer to floor ms albeit slowly improved patient was discharged to with plans to complete day course primaquine and clindamycin allergy to bactrim concern for pancreatitis with pentamidine patient was then admitted to on for altered mental status per dc summary pt had done well and his ms had improved since his dc from such that he was conversant with family on the pt was found unresponsive febrile sent to intubated for resp failure abg on while intubated lac cxr revealed patchy air space opacities cw multifocal pna pt underwent bronchoscopy with witnessed aspiration during the procedure and was started on cefepimevancomycinmetronidazole pcp smears were neg but sputum cx grew out extubated h post admission he was also started on fragmin to histor dvt he was transferred from to for continuit his care followed by doctor per dw family nights prior to this admission pt was sitting up conversant he then had slow decompensation over the next day and was brought back into for eval and treatment coronary artery disease status post mi years ago hypertension hyperlipidemia djd in the cspine depression histor dvt shingles recently finished treatment gliomatosis cerebri grade ii astroca the left temporal lobe with extension into the optic chiasm and left optic nerve diagnosed in after presentation with retrobulbar ischemic optic neuritis status post open brain biopsy status post xrt and low dose temodar repeat mri stable status post high dose temodar landfill workerretired pky histor smoking quit yrs agodenies etohdoctorgs noncontributory physical exam pe t bp p r ra gen arousable to pain opens eyes follows simple commands heent pupil reactive fixed at mm white confluent lesion roof mouth small linear mucosal tear upper side lungs bl rhonchi anteriorly cv regular ss no mrg abd distended soft bs not obviously tender ext no le edema just removed stockings skin no rashes lines ngt pegj tube pertinent results past cultures sputum gram stain gram coccioropharyngeal flora cmv vl negative hsv vzv direct antigen negative toxo igg positive igm negative croccal antigen negative bal pcp respiratory viruses adeno parainfluenza influenza rsv legionella afb all negative cultures from this admission urine ngtd blood ngtd blood ngtd csfspinal fluid ngtd gram stain final no polphonuclear leukocytes seen no microorganisms seen csfspinal fluid cryptococcal antigen not detected urine ngtd sputum pmns and epithelial cellsx field per x field gram positive cocci in pairs and clusters per x field gram positive cocci in pairs and chains grew out urine ngtd blood ngtd blood ngtd urine ngtd laboratory data pm blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood wbc rbc hgb hct mcv mch mchc rdw plt ct am blood fdp am blood fibrino am blood ret aut am blood glucose urean creat na k cl hco angap pm blood glucose urean creat na k cl hco angap am blood ldldh totbili dirbili indbili pm blood alkphos amylase totbili am blood lipase pm blood lipase am blood ckmb ctropnt am blood ckmb ctropnt pm blood ckmb ctropnt am blood ckmb mtwice day ctropnt am blood calcium phos pm blood albumin calcium phos am blood osmolal am blood cortsol am blood cortsol am blood phen phenyfrpnd pm cerebrospinal fluid csf wbc rbc polys lymphs monos pm cerebrospinal fluid csf totprot glucose brief hospital course yo with histor astroca dvt recent admission for pcp pna requiring intubation transferred from withdx aspiration pna after he was found unresponsive febrile pt was intubated at for his asp pna extubated h later transferred to for further care his course at was complicated by ugib from pegj tube erosion into duodenum intubation for egd and fast extubation probable nonconvulsive seizures ms change initially the patients poor mentation was felt to be resolving infection asp pna in the setting low threshold for confusion because his underlying astroca he was continued on antibiotics see below imaging showed that his astroca had not advanced and was not in and itself the cause for his decreased mental capacity the possibility was also raised that there was contribution to his confused state from the steroids that the patient received on his arrival to unclear reason why possibly initial concern that ms change was tumor progression no explanation in dc summary the patient had been on high dose steroids prior to his previous admission but these had been tapered over approx three wks and his mental status had improved several stim tests during this admission showed the patients adoctornal function to be intact he was continued on low dose dexamethasone qday while in house patient was evaluated for nonconvulsive seizure disorder by eeg with prior study showing subcortical dysfunction but not clear evid sz activity first stud this admission showing cortical and subcortical dysfunction no clear evid sz second eeg from this admit on showed paroxysmal semirhythmic epileptiform activiter both frontal regions the patient was started on dilantin and keppra at this time final eeg during this admit on showed an area cortical hypersynchrony likely to be focus sz activity though no such sz activity was observed during the study the patients ms seemed to improve slightly as his dilantin and keppra doses were increased though eventually he plateaued in state minimal responsiveness aspiration pneumoniapneumonitis the patient had at least two likely aspiration events one prior to his admission to when he was found at with tube feeds all over his clothing and second witnessed event during bronchoscopy at pt had completed d course clindaprimaquine and pcp smear was negative at from sputum was felt to be likely colonization given recent hospitalizations and rehab stay patients respiratory failure was felt to be most likely to pneumonitis or bacterial pneumonia after admission he was continued on cefipime started at and flagyl was added given witnessed aspiration at to cover asp flora vancomycin was started to cover from sputum id was consulted rec lp atovaquone ppx for pcp developed that was presumed from cefepime so this was stopped initial series cxrs showed stable diffuse patchacification the right lung patients antibiotics were changed multiple times during his admission on transfer from the patient was on cefepime and levaquin immediately the levaquin was dcd and flagyl was added then on pt was noted to have reticular trunk and thighs thought to be doctorg cefepime was dcd and pt was started back on levofloxacin again instead patient was on the following abx while in house vanc d levaquin d clinda d flagyl d patient was off all antibiotics for three days then noted to have lll asp pna vs pneumonitis on and so levaquin and flagyl were started again on dvt neg cta and lenis here during previous admission fragmin started at was dcd on admission and pt maintained on prophylactic dose heas needed then patients lle was noted to be swollen on so lenis were done that showed bl dvts in superficial and common femoral veins patient was started on heas neededgtt anticoagulation he was then transitioned to lovenox but both were stopped in the setting hct doctorp shortly thereafter with bleeding from his pegj site as patient was poor candidate for anticoagulation ir placed an ivc filter patient was later noted to have lue dvt but was not anticoagulated for this given his continued slow hct doctorp and likely continued ugib bleed if his hct stabilizes in the future and he can be reegd anticoagulation might be indicated however this is unlikely to happen in the near term given his continued gi bleeding and hct doctorp astroca repeat ct scan showed large mass in temporal lobe extending into frontal lobe no significant change since prior study with regard to the mass effect on lateral ventricle no evid brainstem compression patients astroca was not felt to have changed on this admission and as result his symptoms were not felt to be directly related to this lesion it was felt that his tumor may have lowered his threshold for becoming confused somnolent but that either infection or seizures were the cause his altered mental status patient was continued on low dose dexamethasone as well as dilantin and keppra gi bleed the patient had peg placed on in the setting anticoagulation he then began to have melena external bleeding from his pegj site and received transfusions prbc without adequate hct increase he was transferred to the for intubation for the purposes preforming an egd egd showed bleeding around the pegj and string associated with clot and fresh blood in the duodenum the pegj tube was removed by ir over wire and peg tube was placed following the procedure the patient had an ivc filter placed as he has had dvt and given his bleeding anticoagulation was not an option he continued to have doctorpping hct after the procedure and was transfused several more units prbcs over the remainder his admission he was noted to still have gastrografinpositive residuals on in the setting spoke with gi at that point who said that they would not reegd and reintubate the patient unless his bleeding became brisk again they rec to continue ppi supported with transfusions as necessary patient required total units before egd units while in for egd then units after back on the floor for slow hct doctorp with guaiac positivity medications on admission meds on admission senna docusate acetaminophen sertraline qday albuterol neb qh prn levetiracetam dexamethasone iv qh esomeprazole qday atovaquone qday dalteas neededu sc bid metoprolol tid ipratroprium neb once dayvancomycin g qh plan to complete d course cefepime g qh plan to complete d course discharge medications ipratropium bromide solution sig one nebulizer treatment inhalation qh every hours nystatin unitml suspension sig five ml po once day times day atovaquone ml suspension sig five ml po bid times day ml acetaminophen tablet sig tablets po qh every to hours as needed metoprolol tartrate tablet sig one tablet po bid times day dexamethasone tablet sig two tablet po daily daily docusate sodium ml liquid sig ten ml po bid times day senna tablet sig one tablet po bid times day lactulose g ml syrup sig thirty ml po bid times day levetiracetam tablet sig three tablet po bid times day phenn ml suspension sig eight ml po qam qpm ml phenn ml suspension sig twelve ml po qnoon albuterol sulfate solution sig one treatment treatment inhalation qh every hours heas neededflush picc unitsml ml iv dailyprn ml ns followed by ml unitsml heas needed units heas needed each lumen daily and as needed inspect site every shift sodium chloride flush ml iv dailyprn peripheral iv inspect site every shift sodium chloride flush ml iv dailyprn picc please flush picc line with ns inspect site every shift heas neededflush picc unitsml ml iv dailyprn ml ns followed by ml unitsml heas needed units heas needed each lumen daily and as needed inspect site every shift pantoprazole sodium recon soln sig one recon soln intravenous qh every hours discharge disposition home with facility healthcare discharge diagnosis altered mental status aspiration pneumonia nonconvulsive seizures upper gi bleed from pegj tube erosion hypernatremia constipation discharge condition stable though with continued decreased level responsiveness discharge instructions please continue to take all medications as prescribed and to follow up with doctor and the rest your healthcare team if you develop worsening shortness breath high residuals your tube feeds bleeding from anywhere black stool or anher concerning findings please call doctor office immediately for instructions on how to be reevaluated followup instructions please touch base with doctor in the week after your discharge to schedule follow up appointments'

In [2]:
# sentence = 'patient recorded as having no known to doctorgs attending chief complaint renal mass major surgical or invasive procedure right laparoscopic radical nephrectomy yf with cad and esrd on hd with nearly cm left renal mass concerning for malignancy diabetes type cb retinopathy neuropathy pt not currently taking meds due to insurance issue reactive airway disease asthma py smoking and likely copd depression histor pulmonary nodules consistent with calcified granuloma menorrhagia hypertension poorly controlled histor hypertensive urgency hypercholesterolemia chronic lower back pain cri most recent cr values in the s thd mass never followed up with biopsy osteoporosis left renal mass concerning for rcc but never followed up for biopsy chronic anemia thought ckd not on epo mgus chf tte demonstrated moderate to severe global left ventricular hinesis lvef moderate mr and moderate pulmonary artery hypertension lives with roomate in in apt she grew up with her family as carnival worker and traveled with them illiterate smokes ppd has pack yr history no etohillicits on ssi currently only family support seems to be her in ny multiple family members with dm mi cva uncle and two cousins had kidney disease requiring dialysis mother with breast cancer physical exam avss gen nad abd soft nt nd incisions cleandoctorintact brief hospital course patient was admitted to after undergoing laparoscopic right radical nephrectomy no concerning intraoperative events occurred please see operative note for details the patient received perioperative antibiotic prophylaxis the patient was transferred to the icu in stable condition for monitoring due to extensive comorbidities and not for any intraoperative complication on pod pain was well controlled on pca htorted for urine output cchour provided with pneumoboots and incentive spirometry for prophylaxis and ambulated once she was seen by the cardiology and the nephrology and underwent hd while int he icu on pod the patient was restarted on home medications basic metabolic panel and complete blood count were checked pain control was transitioned from pca to oral analgesics diet was advanced to clearstoast and crackers diet on pod foley was removed without difficulty and diet was advanced as tolerated prior to discharge the patient underwent ct scan with contrast the chest to evaluate pulmonary nodules seen on cxr she underwent hd after this the remainder the hospital course was relatively unremarkable the patient was discharged in stable condition eating well ambulating independently voiding without difficulty and with pain control on oral analgesics on exam incision was clean doctor and intact with no evidence hematoma collection or infection the patient was given explicit instructions to followup in clinic with doctor in weeks and to also call for followup with nephrology and cardiology discharge medications docusate sodium capsule sig one capsule po bid times day disp capsules refills acetaminophen tablet sig two tablet po qh every hours asas needed tablet chewable sig one tablet chewable po daily daily omeprazole capsule delayed releasee sig one capsule delayed releasee po daily daily disp capsule delayed releasee refills lisinopril tablet sig two tablet po daily daily disp tablets refills simvastatin tablet sig two tablet po daily daily disp tablets refills complexvitamin cfolic acid capsule sig one cap po daily daily calcium acetate capsule sig two capsule po tid withmeals times day with meals htormorphone tablet sig tablets po every four hours as needed for pain disp tablets refills gabapentin capsule sig two capsule po once dayeach hemodialysis disp capsules refills metoprolol succinate tablet sustained release hr sig one tablet sustained release hr po once day disp tablet sustained release hrs refills lasix tablet sig one tablet po once day disp tablets refills discharge disposition home discharge diagnosis renal mass discharge condition stable discharge instructions you may shower but do not bathe swim or immerse your incision do not lift anything heavier than phone book pounds or doctorve until you are seen by your urologist in followup tylenol should be your first line pain medication narcotic pain medication has been prescribed for breakthough pain replace tylenol with narcotic pain medication max daily tylenol dose is gm note that narcotic pain medication also contains tylenol acetaminophen do not doctorve or doctornk alcohol while taking narcotics colace has been prescribed to avoid post surgical constipation and constipation related to narcotic pain medication discontinue if loose stool or diarrhea develops resume all your home medications except hold nsaid asas needed and ibuprofen containing products such as advil & motrin until you see your urologist in followup call your urologists office today to scheduleconfirm your followup appointment in weeks and if you have any questions if you have fevers vomiting or increased redness swelling or discharge from your incision call your doctor or go to the nearest er do not eat constipating foods for weeks doctornk plent fluids followup instructions please call doctor office for followup please call your cardiologist for followup completed by'

In [3]:
model_path = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract' #'emilyalsentzer/Bio_Discharge_Summary_BERT'

In [4]:
tokenizer = BertTokenizer.from_pretrained(model_path, truncation=True)

In [5]:
pretrain_path = '../src/new_prepreoceed_model/pubmedbert_1ep_3e-05lr_1616bt_nosampler_noextparams_yessch_notlast_can.pt'

In [6]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel

class BERTClass(nn.Module):

        def __init__(self, model_path, drpout):
                super(BERTClass, self).__init__()
                self.model = AutoModel.from_pretrained(model_path, return_dict = True)
                self.l3 = torch.nn.Dropout(drpout)
                self.l4 = torch.nn.Linear(768, 2)
                # nn.init.normal_(self.l4.weight, std=0.02)
                nn.init.xavier_normal_(self.l4.weight)


        def forward(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):

                outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
                pooled_output = outputs[1]
                pooled_output = self.l3(pooled_output)
                logits = self.l4(pooled_output)
                outputs = (logits,) + outputs[2:] 

                if labels is not None:
                        loss_fct = nn.CrossEntropyLoss()
                        loss = loss_fct(logits.view(-1, 2), labels.view(-1))
                        outputs = (loss,) + outputs

                return outputs

In [7]:
model = BertForSequenceClassification.from_pretrained(model_path, return_dict = True, num_labels = 2)
# model = BERTClass(model_path, 0.1)
# model.to(device)
model.load_state_dict(torch.load(pretrain_path))

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

<All keys matched successfully>

In [8]:
# model

In [9]:
def compute_bert_outputs(model_bert, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_bert.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
    else:
        head_mask = [None] * model_bert.config.num_hidden_layers

    encoder_outputs = model_bert.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    pooled_output = model_bert.pooler(sequence_output)
    outputs = (sequence_output, pooled_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)   

In [10]:
class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_bert_outputs(self.model.bert, embeddings)
        pooled_output = outputs[1]
        
        pooled_output = self.model.dropout(pooled_output)
        logits = self.model.classifier(pooled_output)           # [16, 768] --> [16, 2] assuming no of labels is 2
        
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)  # [16, 2] --> [16, 1]

In [11]:
bert_model_wrapper = BertModelWrapper(model)

In [12]:
ig = IntegratedGradients(bert_model_wrapper)
# ig = InputXGradient(bert_model_wrapper)

In [13]:
vis_data_records_ig = []

def interpret_sentence(model_wrapper, sentence, label=1):

    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True, max_length=512, padding='max_length', truncation=True)])
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    
    # predict
    pred = model_wrapper(input_embedding).item()
    pred_ind = round(pred)

    # compute attributions and approximation delta using integrated gradients
    attributions_ig, delta = ig.attribute(input_embedding, n_steps=45, return_convergence_delta=True)

    print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].numpy().tolist())    
    attributions = add_attributions_to_visualizer(attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    
    return tokens[1:-1], attributions  

In [14]:
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    # mean attributions
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().numpy()
    
    # l2 norm
    # attributions = attributions.norm(p=1, dim=-1).squeeze(0)
    # attributions = attributions.detach().numpy()
    
    attributions[0]=0
    attributions[-1]=0
    attributions /= max(attributions)
    
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions[1:-1],
                            pred,
                            pred_ind,
                            label,
                            "label",
                            attributions.sum(),       
                            tokens[1:-1],
                            delta))
    
    return attributions

In [15]:
# the arguments for VisualizationDataRecord are (in order)

# attribution score
# predicted prob
# true label
# predicted label
# attribution label
# total attribution score
# delta (error from IG method due to number of interpolation steps)

In [16]:
# How should we interpret Attribution Score? 
# What's the difference between Target Label and Attribution Label?

# The magnitude of the attribution score shows the strength / the level of the importance 
# of a feature for a particular selected class that we want to attribute to (aka target). 
# If positive, it means that the feature is positively contributing to particular class 
# (e.g. pulling towards the class that we are attributing to), if negative, it means that 
# it is negatively contributing to the class that we are trying to attributing to 
# If zero - means that the feature doesn't contribute to selected target class


# Prediction Label and Attribution aka Target Label

# e.g. if we predict that something on the image is a dog with a high probability then dog is the predicted class. 
# Now, we can attribute our output to dog, cat or anything else that we want to and that is the 
# attribution aka target class / label. I'll fix the header. Thank you!

In [ ]:
# start ke approach

In [ ]:
# single patient

In [24]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [25]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [26]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [16]:
# tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
# atts = visualization.visualize_text(vis_data_records_ig)

In [ ]:
# second patient 

In [27]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [29]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [ ]:
# end ke approach

In [36]:
# same sentence missclassified, with differetn probabilites // Troubleshooting

In [ ]:
# assigning random tokens negative weights

In [30]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [ ]:
# when the mdoel is not performing well, or missclassifying, 
# it attributes the scores to ranodm tokens while if the model is miscclasifying it attributes to other disease better scores 

In [ ]:
# I had a quick question, We were planning on finalizing apartment in the Burton Ridge apartments. 
# All the apartments in downtown area and area close to 

In [31]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

#### Classified as Positive:

In [17]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

#### classifed as negative

In [17]:
# tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=0)
# atts = visualization.visualize_text(vis_data_records_ig)

#### missclassified as negative

In [18]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

#### let's see

In [19]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=0)
atts = visualization.visualize_text(vis_data_records_ig)

### More data model

#### postive classification

In [20]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

#### missclassified as negative

In [21]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

In [22]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=1)
atts = visualization.visualize_text(vis_data_records_ig)

#### missclassified as positive

In [23]:
tokens, attributions = interpret_sentence(bert_model_wrapper, sentence, label=0)
atts = visualization.visualize_text(vis_data_records_ig)

#### bert viz  (only works when model outputs attention)

In [13]:
# from bertviz import model_view, head_view

In [14]:
# sentence_a = "hospital course complicated by benzodiazepines withdrawal" 
# sentence_b = "he became agitated and anxious with increasing psychosis  He eventually started pulling his intravenous lines and wandering into other patient rooms"

In [18]:
# device = 'cpu'

In [15]:
# model_path = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

In [5]:
# tokenizer = BertTokenizer.from_pretrained(model_path, truncation=True)

In [6]:
# model = BertForSequenceClassification.from_pretrained(model_path, return_dict = True, num_labels = 2)
# # model = BERTClass(model_path, 0.1)
# model.load_state_dict(torch.load("../src/new_prepreoceed_model/pubmedbert_1ep_3e-05lr_1616bt_nosampler_noextparams_yessch_notlast_can.pt"))
# model.eval()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

<All keys matched successfully>

In [86]:
# def show_model_view(model, tokenizer, sentence_a, sentence_b = None, hide_delimiter_attn=False):
#     inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
#     input_ids = inputs['input_ids']
#     if sentence_b:
#         token_type_ids = inputs['token_type_ids']
#         attention = model(input_ids, token_type_ids=token_type_ids)[-1]
#         sentence_b_start = token_type_ids[0].tolist().index(1)
#     else:
#         attention = model(input_ids)[-1]
#         sentence_b_start = None
#     input_id_list = input_ids[0].tolist() # Batch index 0
#     tokens = tokenizer.convert_ids_to_tokens(input_id_list)  
#     if hide_delimiter_attn:
#         for i, t in enumerate(tokens):
#             if t in ("[SEP]", "[CLS]"):
#                 for layer_attn in attention:
#                     layer_attn[0, :, i, :] = 0
#                     layer_attn[0, :, :, i] = 0
#     model_view(attention, tokens, sentence_b_start)

In [88]:
# show_model_view(model, tokenizer, sentence_a, sentence_b, hide_delimiter_attn=False)

In [19]:
# def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
#     inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
#     input_ids = inputs['input_ids'].to(device)
#     if sentence_b:
#         token_type_ids = inputs['token_type_ids'].to(device)
#         attention = model(input_ids, token_type_ids=token_type_ids)[-1]
#         sentence_b_start = token_type_ids[0].tolist().index(1)
#     else:
#         attention = model(input_ids)[-1]
#         sentence_b_start = None
#     input_id_list = input_ids[0].tolist() # Batch index 0
#     tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
#     head_view(attention, tokens, sentence_b_start)

In [22]:
# show_head_view(model, tokenizer, sentence_a, sentence_b)